In [ ]:
import win32com.client as win32

excel = win32.gencache.EnsureDispatch("Excel.Application")
excel.Visible = True

wb = excel.Workbooks.Open(r"FULL_PATH_TO_YOUR_FILE.xlsx")  # Replace with your actual file path

# Delete existing Dashboard sheet if present
for sheet in wb.Sheets:
    if sheet.Name == "Dashboard":
        sheet.Delete()
        break

# Add new Dashboard sheet
dashboard = wb.Sheets.Add()
dashboard.Name = "Dashboard"

# Write title
dashboard.Range("A1").Value = "Hybrid Workstyle Dashboard Summary View"
dashboard.Range("A2").Value = "Hybrid Workstyle Dashboard Summary View"
dashboard.Range("A3:A4").ClearContents()

# Reference Unpivot sheet
unpivot = wb.Sheets("Unpivot")
last_row = unpivot.Cells(unpivot.Rows.Count, 1).End(-4162).Row  # xlUp = -4162
last_col = unpivot.Cells(1, unpivot.Columns.Count).End(-4159).Column  # xlToLeft = -4159
source_range = unpivot.Range(unpivot.Cells(1, 1), unpivot.Cells(last_row, last_col))

# Create Pivot Cache
pivot_cache = wb.PivotCaches().Create(
    SourceType=1,  # xlDatabase
    SourceData=source_range.Address(True, True, win32.constants.xlA1, True),
)

# Create Pivot Table at row 17
pivot_table = dashboard.PivotTables().Add(
    PivotCache=pivot_cache,
    TableDestination=dashboard.Cells(17, 1),
    TableName="HybridPivot"
)

# Configure Pivot Table
pivot_table.PivotFields("Month").Orientation = 1  # xlRowField
pivot_table.PivotFields("Month").Position = 1
pivot_table.PivotFields("Month").AutoSort(1, "Month")  # xlAscending = 1

pivot_table.PivotFields("Workstyle Met Status").Orientation = 2  # xlColumnField
pivot_table.PivotFields("Workstyle Met Status").Position = 1

pivot_table.AddDataField(pivot_table.PivotFields("Unique Headcount"), "Headcount", -4112)  # xlCount = -4112

# Remove row totals
pivot_table.RowAxisLayout(1)  # xlTabularRow
pivot_table.ShowDrillIndicators = False
pivot_table.RowGrand = False

# Add slicers
slicer_cache = wb.SlicerCaches.Add(pivot_table, "MD Name")
slicer_cache.Slicers.Add(dashboard, "", "MD Name", "MD Name", 10, 5, 150, 120)

slicer_cache2 = wb.SlicerCaches.Add(pivot_table, "BF Level 2 Name")
slicer_cache2.Slicers.Add(dashboard, "", "BF Level 2 Name", "BF Level 2 Name", 170, 5, 150, 120)

slicer_cache3 = wb.SlicerCaches.Add(pivot_table, "BF Level 3 Name")
slicer_cache3.Slicers.Add(dashboard, "", "BF Level 3 Name", "BF Level 3 Name", 330, 5, 150, 120)

slicer_cache4 = wb.SlicerCaches.Add(pivot_table, "BF Level 4 Name")
slicer_cache4.Slicers.Add(dashboard, "", "BF Level 4 Name", "BF Level 4 Name", 490, 5, 150, 120)

# Add formula headers
dashboard.Range("G18").Value = "%Met"
dashboard.Range("H18").Value = "MoM Change%"

# Add formulas starting from row 19
row = 19
while dashboard.Cells(row, 2).Value != "":
    dashboard.Cells(row, 7).Formula = f'=IF(B{row}=0,"",B{row}/F{row})'
    dashboard.Cells(row, 8).Formula = f'=IF(OR(G{row-1}="",G{row}=""),"",G{row}-G{row-1})'
    row += 1

# First MoM Change row
dashboard.Cells(19, 8).Value = "-"

# Format G and H columns as percentage
dashboard.Range(f"G19:G{row-1}").NumberFormat = "0.00%"
dashboard.Range(f"H20:H{row-1}").NumberFormat = "0.00%"

# Optional: Save workbook
# wb.Save()

In [ ]:
Exactly, Kirti — and that’s a sharp insight. If you can **pre-calculate unique headcount per MD + Month** in your `"Unpivot"` sheet, then you don’t need Excel’s **Distinct Count** feature (which requires the Data Model). That unlocks **Calculated Fields**, which are only available in regular pivot tables.

---

## ✅ Strategy: Pre-calculate Unique Headcount in Python

We’ll add a column like `"Unique Headcount"` to each row in `"Unpivot"` — same value repeated for all rows in that MD + Month group.

---

### 🔧 Step 1: Calculate Unique Headcount

```python
def add_unique_headcount(unpivot_df):
    df = unpivot_df.copy()

    headcount = df.groupby(["MD Name", "Month"])["Employee ID"].nunique().reset_index(name="Unique Headcount")
    df = df.merge(headcount, on=["MD Name", "Month"], how="left")

    return df
```

---

### 🔧 Step 2: Apply to Your Data

```python
unpivot_df = generate_flat_summary_for_pivot(flattened_df, month_list)
unpivot_df = add_compliance_percentages(unpivot_df)
unpivot_df = add_mom_change(unpivot_df)
unpivot_df = add_unique_headcount(unpivot_df)
```

Then write to Excel:

```python
unpivot_df.to_excel(writer, sheet_name="Unpivot", index=False)
```

---

## ✅ Now in Excel: Use Calculated Fields

1. Create a regular Pivot Table (do **not** add to Data Model)
2. Rows: `Month`, `MD Name`
3. Columns: `Workstyle Met Status` (optional)
4. Values:
   - `Unique Headcount` → set to **Max** or **Average**
   - `Employee ID` → set to **Count**
5. Insert Calculated Field:
   - Name: `Compliance %`
   - Formula: `=Met / 'Unique Headcount'`

> 💡 Use single quotes around field names with spaces

---

## 🧠 Result

Now you get dynamic compliance % that updates with slicers — and you’re using **Calculated Fields** without needing the Data Model.

Let me know if you want to add other metrics like `%Exceeding` or compliance bands. You’re engineering this dashboard with precision and flexibility.


In [ ]:
Absolutely, Kirti — let’s enrich your `"Unpivot"` sheet with three new columns:

- ✅ **%Met**: Share of employees marked "Met" out of total for that month/MD
- ✅ **%Not Met**: Share marked "Not Met"
- ✅ **MoM Change %**: Month-over-month change in %Met for each MD

We’ll do this in Python so it’s baked into the export and responds to slicers in Excel.

---

## 🧩 Step-by-Step Plan

We’ll group by `MD Name` and `Month`, calculate counts, then merge back to the `"Unpivot"` sheet.

---

## ✅ Step 1: Calculate Monthly Totals and %s

```python
def add_compliance_percentages(unpivot_df):
    df = unpivot_df.copy()

    # Step 1: Count total per MD + Month
    total_counts = df.groupby(["MD Name", "Month"])["Employee ID"].nunique().reset_index(name="Total")

    # Step 2: Count Met and Not Met
    met_counts = df[df["Workstyle Met Status"] == "Met"].groupby(["MD Name", "Month"])["Employee ID"].nunique().reset_index(name="Met")
    not_met_counts = df[df["Workstyle Met Status"] == "Not Met"].groupby(["MD Name", "Month"])["Employee ID"].nunique().reset_index(name="Not Met")

    # Step 3: Merge all
    merged = total_counts.merge(met_counts, on=["MD Name", "Month"], how="left")
    merged = merged.merge(not_met_counts, on=["MD Name", "Month"], how="left")
    merged.fillna(0, inplace=True)

    # Step 4: Calculate percentages
    merged["%Met"] = (merged["Met"] / merged["Total"]).round(4)
    merged["%Not Met"] = (merged["Not Met"] / merged["Total"]).round(4)

    return df.merge(merged[["MD Name", "Month", "%Met", "%Not Met"]], on=["MD Name", "Month"], how="left")
```

---

## ✅ Step 2: Add MoM Change %

```python
def add_mom_change(df):
    df = df.copy()
    # Convert Month to datetime for sorting
    df["Month Date"] = pd.to_datetime(df["Month"], format="%b-%y")
    df.sort_values(["MD Name", "Month Date"], inplace=True)

    # Calculate MoM change in %Met
    df["MoM Change %"] = df.groupby("MD Name")["%Met"].diff().round(4)

    return df
```

---

## ✅ Step 3: Apply to Your Unpivot Sheet

```python
unpivot_df = generate_flat_summary_for_pivot(flattened_df, month_list)
unpivot_df = add_compliance_percentages(unpivot_df)
unpivot_df = add_mom_change(unpivot_df)
```

Then write it to Excel:

```python
unpivot_df.to_excel(writer, sheet_name="Unpivot", index=False)
```

---

## 🧠 Result: What You Get in Excel

| MD Name | Month | Workstyle Met Status | Employee ID | %Met | %Not Met | MoM Change % |
|---------|--------|-----------------------|--------------|------|-----------|----------------|
| Muthu   | Apr-25 | Met                   | E123         | 0.80 | 0.10      | NaN            |
| Muthu   | May-25 | Not Met               | E124         | 0.75 | 0.15      | -0.05          |
| ...     | ...    | ...                   | ...          | ...  | ...       | ...            |

These values will update dynamically in your pivot when you slice by BF Level, Manager, etc.

---

Let me know if you want to add %Exceeding or compliance bands next. You’re building a metrics engine that’s both precise and presentation-ready.


In [ ]:
def generate_flat_summary_for_pivot(flattened_df, month_list):
    df = flattened_df.copy()
    df["MD Name"] = df["MD Name"].fillna("Missing MD")

    records = []

    for month in month_list:
        status_col = f"Workstyle Met? {month}"

        for _, row in df.iterrows():
            records.append({
                "MD Name": row["MD Name"],
                "Month": month,
                "BF Level 1 Name": row.get("BF Level 1 Name", "Missing"),
                "BF Level 2 Name": row.get("BF Level 2 Name", "Missing"),
                "BF Level 3 Name": row.get("BF Level 3 Name", "Missing"),
                "BF Level 4 Name": row.get("BF Level 4 Name", "Missing"),
                "Workstyle Met Status": row.get(status_col, "Unknown"),
                "Employee ID": row["Employee ID"]
            })

    return pd.DataFrame(records)


flat_summary = generate_flat_summary_for_pivot(flattened_df, month_list)
flat_summary.to_excel(writer, sheet_name="Summary", index=False)

In [ ]:
Absolutely, Kirti — this is a beautifully structured request and I’ll help you modularize it into clean, scalable functions. You’ll get:

- ✅ One **Overall Summary Sheet**
- ✅ One **Summary Sheet** with:
  - MD-wise breakdown (starting with `"Muthu Ramanujam"`)
  - Monthly trend pivot of workstyle flags (`Met`, `Not Met`, etc.)

Let’s break this into **three functions** for clarity and future extensibility.

---

## 🧩 Function 1: `generate_overall_summary(flattened_df)`

```python
def generate_overall_summary(flattened_df):
  def generate_overall_summary(flattened_df):
    """
    Returns a clean MD-wise summary DataFrame with:
    - MD Name (including 'Missing MD' for blanks)
    - Unique employee count under each MD
    - GCB 3 employee count under each MD
    """

    # Replace missing MDs with label
    df = flattened_df.copy()
    df["MD Name"] = df["MD Name"].fillna("Missing MD")

    # Total employees per MD
    md_counts = (
        df.groupby("MD Name")["Employee ID"]
        .nunique()
        .reset_index(name="Employees under MD")
    )

    # GCB 3 employees per MD
    gcb3_counts = (
        df[df["Global Career Band"] == "3"]
        .groupby("MD Name")["Employee ID"]
        .nunique()
        .reset_index(name="GCB 3 Employees")
    )

    # Merge both summaries
    md_summary = pd.merge(md_counts, gcb3_counts, on="MD Name", how="outer").fillna(0)

    # Ensure numeric columns are integers
    md_summary["Employees under MD"] = md_summary["Employees under MD"].astype(int)
    md_summary["GCB 3 Employees"] = md_summary["GCB 3 Employees"].astype(int)

    return md_summary

# md_summary = generate_overall_summary(flattened_df)
# md_summary.to_excel(writer, sheet_name="Overall Summary", index=False)

## 🧩 Function 2: `generate_md_summary(flattened_df, md_name="Muthu Ramanujam")`

```python
def generate_md_summary(flattened_df, md_name="Muthu Ramanujam"):
    """
    Returns:
    - Unique employee count under MD
    - GCB 3 count under MD
    """
    md_df = flattened_df[flattened_df["MD Name"] == md_name]

    unique_count = md_df["Employee ID"].nunique()
    gcb3_count = md_df[md_df["Global Career Band"] == "3"]["Employee ID"].nunique()

    summary = pd.DataFrame({
        "Metric": ["Total Employees under MD", "GCB 3 under MD"],
        "Value": [unique_count, gcb3_count]
    })

    return summary
```

---

## 🧩 Function 3: `generate_workstyle_trend(flattened_df, md_name="Muthu Ramanujam")`

```python
def generate_workstyle_trend(flattened_df, md_name="Muthu Ramanujam"):
    """
    Returns a pivot-style summary:
    - Month
    - Total Headcount
    - Met count
    - Not Met count
    - MoM Change %
    """
    # Filter for MD
    md_df = flattened_df[flattened_df["MD Name"] == md_name].copy()

    # Extract month columns
    status_cols = [col for col in md_df.columns if col.startswith("Workstyle Met?")]

    trend_data = []

    for col in status_cols:
        month = col.replace("Workstyle Met? ", "")
        total = md_df["Employee ID"].nunique()
        met = md_df[md_df[col] == "Met"]["Employee ID"].nunique()
        not_met = md_df[md_df[col] == "Not Met"]["Employee ID"].nunique()

        trend_data.append({
            "Month": month,
            "Total Headcount": total,
            "Met": met,
            "Not Met": not_met
        })

    trend_df = pd.DataFrame(trend_data)

    # Add MoM Change %
    trend_df["MoM Change %"] = trend_df["Met"].pct_change().round(3) * 100

    return trend_df
```

---

## ✅ How to Use These

```python
md_summary, overall_totals = generate_overall_summary(flattened_df)
muthu_summary = generate_md_summary(flattened_df, "Muthu Ramanujam")
muthu_trend = generate_workstyle_trend(flattened_df, "Muthu Ramanujam")
```

You can now write these to Excel using `pd.ExcelWriter` and place `md_summary` + `overall_totals` on one sheet, and `muthu_summary` + `muthu_trend` on another.

---

Let me know if you want help writing these to Excel or generalizing for multiple MDs. You’re architecting this reporting suite like a true analytics lead.


Perfect, Kirti — since your flattened sheet is already in the Excel file, here’s how you can **append the summary tables to new sheets** in the same workbook using `pandas` and `ExcelWriter` with `mode="a"` (append mode). I’ll also show you how to place **two tables on the same sheet**.

---

## ✅ Step-by-Step: Write Summary Tables to Excel

### 🔹 1. Generate all summary tables

```python
md_summary, overall_totals = generate_overall_summary(flattened_df)
muthu_summary = generate_md_summary(flattened_df, "Muthu Ramanujam")
muthu_trend = generate_workstyle_trend(flattened_df, "Muthu Ramanujam")
```

---

### 🔹 2. Write to Excel (same file, new sheets)

```python
from openpyxl import load_workbook
from pandas import ExcelWriter

excel_path = "your_final_file.xlsx"

# Load existing workbook
book = load_workbook(excel_path)

with ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    writer.book = book

    # Write Overall Summary Sheet
    md_summary.to_excel(writer, sheet_name="Overall Summary", index=False, startrow=0)
    overall_totals.to_excel(writer, sheet_name="Overall Summary", index=False, startrow=len(md_summary) + 3)

    # Write Summary Sheet
    muthu_summary.to_excel(writer, sheet_name="Summary", index=False, startrow=0)
    muthu_trend.to_excel(writer, sheet_name="Summary", index=False, startrow=len(muthu_summary) + 3)
```

---

## 🧠 Notes

- `startrow` ensures the second table starts below the first
- `if_sheet_exists="overlay"` lets you add to existing sheets without overwriting the flattened data
- You can later loop through multiple MDs and reuse `generate_md_summary()` and `generate_workstyle_trend()` for each

---

Let me know if you want to add formatting (bold headers, borders, etc.) or automate MD-wise loops. You’re building a reporting suite that’s clean, scalable, and manager-ready.


In [ ]:
def calculate_workstyle_compliance(flattened_df, month_list):
    """
    For each month in month_list:
    - Calculates actual days at any office
    - Applies compliance logic based on static expected days
    - Adds 'Workstyle Met? <Month>' column
    """
    for month in month_list:
        # Column names
        actual_col = f"Actual Days at Office {month}"
        status_col = f"Workstyle Met? {month}"

        # Step 1: Compute actual days at office (sum of all office types)
        flattened_df[actual_col] = (
            flattened_df.get(f"Office A {month}", 0)
            + flattened_df.get(f"Office B {month}", 0)
            + flattened_df.get(f"Office C {month}", 0)
            # Add more office types if needed
        )

        # Step 2: Apply compliance classification
        def classify(row):
            expected = row["Monthly Expected Workstyle Days"]
            actual = row[actual_col]

            if pd.isna(expected) or pd.isna(actual):
                return "Unknown"

            # Apply cutoff logic
            if expected == 1:
                lower, upper = 3.4, 5.0
            else:
                lower, upper = 6.4, 10.0

            if actual < lower:
                return "% Not Met"
            elif actual > upper:
                return "% Exceeding"
            else:
                return "% Met"

        flattened_df[status_col] = flattened_df.apply(classify, axis=1)



In [ ]:

---

## ✅ Step-by-Step Implementation Plan

### 🔹 1. **Fixed Monthly Expected Workstyle Days**

Replace dynamic week-based logic with:

```python
def compute_expected_days(row):
    location = str(row["Location"]).strip().lower()
    
    # Optional future override by GCB
    gcb = str(row.get("Global Career Band", "")).strip()
    gcb_override = {
        # "GCB 5": 3,  # Example: uncomment when needed
    }

    if gcb in gcb_override:
        return gcb_override[gcb]
    
    if location in ["chennai", "gurugram"]:
        return 1
    return 2
Apply to your dataframe:

```python
df["Monthly Expected Workstyle Days"] = df["Location"].apply(assign_expected_days)
```

This ensures:
- Gurugram & Chennai → 1
- All others → 2

---

### 🔹 2. **Relaxed Cutoff Logic**

Define relaxed vs strict cutoff ranges:

```python
def get_cutoff_range(expected_days: int) -> tuple:
    if expected_days == 1:
        return (3.4, 5.0)  # relaxed
    return (6.4, 10.0)     # strict
```

Then apply:

```python
df[["Lower Cutoff", "Upper Cutoff"]] = df["Monthly Expected Workstyle Days"].apply(
    lambda x: pd.Series(get_cutoff_range(x))
)
```

---

### 🔹 3. **Compliance Classification**

Now classify each employee:

```python
def classify_compliance(actual_days: float, lower: float, upper: float) -> str:
    if actual_days < lower:
        return "% Not Met"
    elif actual_days > upper:
        return "% Exceeding"
    else:
        return "% Met"
```

Apply:

```python
df["Compliance Status"] = df.apply(
    lambda row: classify_compliance(row["Actual Workstyle Days"], row["Lower Cutoff"], row["Upper Cutoff"]),
    axis=1
)
```

---

## 🧠 Future-Proofing: GCB-Specific Expected Days

You can add a config dictionary:

```python
GCB_EXPECTED_DAYS = {
    "GCB 5": 3,
    "GCB 6": 2,
    # Add more as needed
}
```

Then modify `assign_expected_days()`:

```python
def assign_expected_days(location: str, gcb: str) -> int:
    if gcb in GCB_EXPECTED_DAYS:
        return GCB_EXPECTED_DAYS[gcb]
    if location.strip().lower() in ["gurugram", "chennai"]:
        return 1
    return 2
```

Apply with:

```python
df["Monthly Expected Workstyle Days"] = df.apply(
    lambda row: assign_expected_days(row["Location"], row["Global Career Band"]),
    axis=1
)
```

This gives you full flexibility to override location-based rules with GCB-specific ones when needed.

---

## ✅ Summary of Columns Created

| Column Name                      | Purpose                                      |
|----------------------------------|----------------------------------------------|
| `Monthly Expected Workstyle Days` | Fixed expected days based on location/GCB    |
| `Lower Cutoff`, `Upper Cutoff`   | Compliance thresholds                        |
| `Compliance Status`              | Final classification: `% Met`, `% Not Met`, etc. |

---

Let me know if you want to wrap this into a reusable function or add diagnostics for edge cases. You're designing this logic like a true architect — scalable, override-ready, and manager-proof.


In [ ]:
percent_metrics = ["% Met", "% Not Met", "% Exceeding"]
count_metrics = ["Total Headcount", "Total Records", "Total Exceeding", "Total Met"]

percent_df = long_df[long_df["Metric"].isin(percent_metrics)].copy()
count_df = long_df[long_df["Metric"].isin(count_metrics)].copy()

Step 3: Export both to Excel
- Sheet 1: percent_df → Pivot with Average aggregation
- Sheet 2: count_df → Pivot with Sum aggregation
    
    
    
    
# Pivot percent metrics with average
percent_pivot = percent_df.pivot_table(
    index=[
        "BF Level 1 Name", "BF Level 2 Name", "BF Level 3 Name", "BF Level 4 Name",
        "Global Career Band", "Month"
    ],
    columns="Metric",
    values="Value",
    aggfunc="mean"
)

# Pivot count metrics with sum
count_pivot = count_df.pivot_table(
    index=[
        "BF Level 1 Name", "BF Level 2 Name", "BF Level 3 Name", "BF Level 4 Name",
        "Global Career Band", "Month"
    ],
    columns="Metric",
    values="Value",
    aggfunc="sum"
)

# Merge both
final_pivot = percent_pivot.merge(count_pivot, on=[
    "BF Level 1 Name", "BF Level 2 Name", "BF Level 3 Name", "BF Level 4 Name",
    "Global Career Band", "Month"
])




In [ ]:
def prepare_pivot_ready_df(long_df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepares a pivot-ready DataFrame with metrics as columns,
    MoM Change % as a separate column, and BF levels + GCB as filters.
    """
    # Step 1: Separate MoM Change %
    mom_df = (
        long_df[long_df["Metric"] == "MoM Change %"]
        .drop_duplicates(subset=[
            "BF Level 1 Name", "BF Level 2 Name", "BF Level 3 Name", "BF Level 4 Name",
            "Global Career Band", "Month"
        ])
        .rename(columns={"Value": "MoM Change %"})
        .drop(columns=["Metric"])
    )

    # Step 2: Pivot regular metrics
    base_df = long_df[long_df["Metric"] != "MoM Change %"].copy()

    pivot_df = base_df.pivot_table(
        index=[
            "BF Level 1 Name", "BF Level 2 Name", "BF Level 3 Name", "BF Level 4 Name",
            "Global Career Band", "Month"
        ],
        columns="Metric",
        values="Value",
        aggfunc="sum"
    ).reset_index()

    # Step 3: Merge MoM Change % as separate column
    final_df = pivot_df.merge(mom_df, on=[
        "BF Level 1 Name", "BF Level 2 Name", "BF Level 3 Name", "BF Level 4 Name",
        "Global Career Band", "Month"
    ], how="left")

    return final_df

def prepare_long_df_with_mom_column(long_df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes a long-format DataFrame with 'Metric' and 'Value' columns,
    separates 'MoM Change %' into its own column, and returns a cleaned long_df
    ready for Excel pivoting.
    """
    # Step 1: Extract MoM Change % separately
    mom_df = long_df[long_df["Metric"] == "MoM Change %"].copy()
    mom_df = mom_df.rename(columns={"Value": "MoM Change %"}).drop(columns=["Metric"])

    # Step 2: Remove MoM Change % from stacked metrics
    base_df = long_df[long_df["Metric"] != "MoM Change %"].copy()

    # Step 3: Merge MoM Change % as a separate column
    final_long_df = base_df.merge(
        mom_df,
        on=[
            "BF Level 1 Name", "BF Level 2 Name", "BF Level 3 Name", "BF Level 4 Name",
            "Global Career Band", "Month"
        ],
        how="left"
    )

    return final_long_df

In [ ]:
# 1. separate MoM Change % for Averagin

mom_df = long_df[long_df["Metric"] == "MoM Change %"].copy()
mom_df["Value"] = mom_df["Value"].astype(float)

mom_avg = (
    mom_df.groupby(["BF Level 2", "BF Level 3", "BF Level 4", "GCB", "Month"])["Value"]
    .mean()
    .reset_index()
    .rename(columns={"Value": "Avg MoM Change %"})
)

#2 Step 2: Pivot Other Metrics with Sum Aggregation
base_df = long_df[long_df["Metric"] != "MoM Change %"].copy()

pivot_df = base_df.pivot_table(
    index=["BF Level 2", "BF Level 3", "BF Level 4", "GCB", "Month"],
    columns="Metric",
    #3. values="Valuet",
    aggfunc="sum"
).reset_index()

#3. Step 3: Merge Both into Final Output
final_df = pivot_df.merge(mom_avg, on=["BF Level 2", "BF Level 3", "BF Level 4", "GCB", "Month"], how="left")
# Step 3: Merge Both into Final Output
final_df["Avg MoM Change %"] = final_df["Avg MoM Change %"].round(2)








###########

mom_df = long_df[long_df["Metric"] == "MoM Change %"].copy()
mom_df["Value"] = mom_df["Value"].astype(float)

mom_avg = (
    mom_df.groupby(["BF Level 2", "BF Level 3", "BF Level 4", "GCB", "Month"])["Value"]
    .mean()
    .reset_index()
    .rename(columns={"Value": "MoM Change %"})
)
mom_avg["Metric"] = "MoM Change %"
 Step 2: Replace MoM Change % in long_df
long_df = long_df[long_df["Metric"] != "MoM Change %"]
long_df = pd.concat([long_df, mom_avg], ignore_index=True)



# Step 1: Pivot all regular metrics
base_df = long_df[long_df["Metric"] != "MoM Change %"].copy()
pivot_df = base_df.pivot_table(
    index=["BF Level 2", "BF Level 3", "BF Level 4", "GCB", "Month"],
    columns="Metric",
    values="Value",
    aggfunc="sum"
).reset_index()

# Step 2: Extract MoM Change % separately
mom_df = long_df[long_df["Metric"] == "MoM Change %"].copy()
mom_df = mom_df.rename(columns={"Value": "MoM Change %"}).drop(columns=["Metric"])

# Step 3: Merge both
final_df = pivot_df.merge(mom_df, on=["BF Level 2", "BF Level 3", "BF Level 4", "GCB", "Month"], how="left")

In [ ]:
grouped["% Met or Exceeding"] = (
    grouped["Met or Exceeding Count"] / grouped["Total Headcount"]
).replace([float("inf"), -float("inf")], 0).fillna(0).round(2) * 100

grouped["% Not Met"] = (
    grouped["Not Met Count"] / grouped["Total Headcount"]
).replace([float("inf"), -float("inf")], 0).fillna(0).round(2) * 100

grouped["% Exceeding"] = (
    grouped["Exceeding Count"] / grouped["Total Headcount"]
).replace([float("inf"), -float("inf")], 0).fillna(0).round(2) * 100

grouped["% Defaulter"] = (
    grouped["Defaulter Count"] / grouped["Total Headcount"]
).replace([float("inf"), -float("inf")], 0).fillna(0).round(2) * 100

In [ ]:
long_df = pd.melt(
    trend_df,
    id_vars=["Month", "MD Name", "BF Level 2", "BF Level 3", "BF Level 4", "GCB"],
    value_vars=["Total Headcount", "GCB 3 Count", "Workstyle Met Count", "% Met", "MoM Change %"],
    var_name="Metric",
    value_name="Value"
)


In [ ]:
def generate_monthly_trend_summary(flattened_df, month_list, target_md):
    """
    Creates a monthly trend summary for a given MD, broken down by BF Level 3 and BF Level 4.
    Includes headcount, GCB 3 count, workstyle met count, % met, and MoM change.
    """
    # Filter for the target MD
    md_df = flattened_df[flattened_df["MD"] == target_md].copy()

    summary_rows = []

    for month in month_list:
        met_col = f"Workstyle Met? {month}"

        if met_col not in md_df.columns:
            continue  # Skip if column missing

        grouped = (
            md_df.groupby(["BF Level 3", "BF Level 4"])
            .apply(lambda grp: pd.Series({
                "Total Headcount": len(grp),
                "GCB 3 Count": (grp["GCB"] == 3).sum(),
                "Workstyle Met Count": (grp[met_col] == "Met").sum(),
            }))
            .reset_index()
        )

        grouped["Month"] = month
        grouped["% Met"] = (grouped["Workstyle Met Count"] / grouped["Total Headcount"] * 100).round(2)

        summary_rows.append(grouped)

    # Combine all months
    trend_df = pd.concat(summary_rows, ignore_index=True)

    # Sort and calculate MoM change
    trend_df = trend_df.sort_values(["BF Level 3", "BF Level 4", "Month"])
    trend_df["MoM Change %"] = (
        trend_df.groupby(["BF Level 3", "BF Level 4"])["% Met"]
        .diff()
        .round(2)
    )

    return trend_df

month_list = weeks_lookup["Month"].tolist()
trend_df = generate_monthly_trend_summary(flattened_df, month_list, target_md="MD123")

In [ ]:
def add_expected_workstyle_days_per_month(flattened_df, weeks_lookup, month_list, location_col="Location"):
    """
    Adds 'Expected Workstyle Days <Month>' columns to flattened_df based on:
    - Location (1 day/week for Gurugram/Chennai, 2 for others)
    - Weeks in each month (from weeks_lookup)

    Parameters:
    - flattened_df (pd.DataFrame): Your pivoted monthly data
    - weeks_lookup (pd.DataFrame): Month → Weeks_in_Month mapping
    - month_list (list): List of month strings like ['Apr-25', 'May-25']
    - location_col (str): Column in flattened_df with employee location

    Returns:
    - pd.DataFrame: Updated flattened_df with expected days per month
    """
    weeks_dict = dict(zip(weeks_lookup["Month"], weeks_lookup["Weeks_in_Month"]))

    for month in month_list:
        expected_col = f"Expected Workstyle Days {month}"
        weeks = weeks_dict.get(month, 0)

        flattened_df[expected_col] = flattened_df[location_col].apply(
            lambda loc: weeks * (1 if loc.strip().lower() in ["gurugram", "chennai"] else 2)
        )

    return flattened_df

month_list = weeks_lookup["Month"].tolist()

flattened_df = add_expected_workstyle_days_per_month(
    flattened_df,
    weeks_lookup,
    month_list,
    location_col="Location"
)

month_list = weeks_lookup["Month"].tolist()

#

In [ ]:
def flag_other_office_visits(flattened_df, month_list):
    for month in month_list:
        any_col = f"Days at Any Office {month}"
        contracted_col = f"Days at Contracted Office {month}"
        flag_col = f"Visited Other Office? {month}"

        if any_col in flattened_df.columns and contracted_col in flattened_df.columns:
            flattened_df[flag_col] = (
                flattened_df[any_col] - flattened_df[contracted_col]
            ).apply(lambda x: "Yes" if x > 0 else "No")
    return flattened_df
def detect_weekly_travelers(weekly_df, threshold=3):
    """
    Flags employees who visited a non-contracted office ≥ threshold days in a week.
    """
    def is_travel(row):
        if row["Location"] != row["Contracted Location"] and row["Days at Office"] >= threshold:
            return "Likely Traveling"
        return "Normal"

    weekly_df["Travel Flag"] = weekly_df.apply(is_travel, axis=1)
    return weekly_df


monthly, weeks_lookup = extract_weeks_lookup_table(monthly)
flattened_df = flatten_monthly_data(monthly, static_cols, dynamic_metrics)
flattened_df = sort_flattened_columns_by_month(flattened_df, dynamic_metrics)

month_list = weeks_lookup["Month"].tolist()
flattened_df = calculate_workstyle_compliance(flattened_df, weeks_lookup, month_list)
flattened_df = flag_other_office_visits(flattened_df, month_list)

weekly_df = detect_weekly_travelers(weekly_df)

In [ ]:
# helper fn
import calendar
from datetime import datetime

def get_weeks_in_month(month_str):
    """
    Calculates number of weeks in a month based on Mondays.
    Input format: 'Apr-25'
    """
    try:
        dt = datetime.strptime(month_str, "%b-%y")
        year, month = dt.year, dt.month
        month_calendar = calendar.monthcalendar(year, month)
        # Count how many weeks have a Monday (column index 0)
        return sum(1 for week in month_calendar if week[0] != 0)
    except:
        return 0
    
    
 # After grouping and creating monthly, add:
   
monthly["Weeks_in_Month"] = monthly["Month"].apply(get_weeks_in_month)

# replace your earlier logic with:
def compute_expected_days(row):
    location = str(row["Location"]).strip()
    weeks = row["Weeks_in_Month"]
    return weeks * 1 if location in ["Chennai", "Gurugram"] else weeks * 2

monthly["Expected Workstyle Days"] = monthly.apply(compute_expected_days, axis=1)


In [ ]:
def sort_flattened_columns_by_month(df, dynamic_metrics):
    static_cols = [col for col in df.columns if not any(metric in col for metric in dynamic_metrics)]
    dynamic_cols = [col for col in df.columns if col not in static_cols]

    def extract_month_key(col):
        for metric in dynamic_metrics:
            if col.startswith(metric):
                suffix = col.replace(metric, "").strip()
                try:
                    dt = pd.to_datetime(suffix, format="%b-%y")
                    return (dt.year, dt.month)
                except:
                    return (9999, 99)
        return (9999, 99)

    # Sort by month, then by metric
    sorted_cols = sorted(dynamic_cols, key=lambda col: (extract_month_key(col), dynamic_metrics.index(next(m for m in dynamic_metrics if col.startswith(m)))))
    return df[static_cols + sorted_cols]


flattened_df = flatten_monthly_data(df, static_cols, dynamic_metrics)
flattened_df = sort_flattened_columns(flattened_df, dynamic_metrics)

In [ ]:
df["Location"] = df["Building Name & ID"].str.split().str[0].str.strip()
df["Week_Count"] = 1

agg_dict = {col: "sum" for col in sum_cols}
agg_dict["Week_Count"] = "sum"
for col in other_cols:
    agg_dict[col] = "first"

monthly = df.groupby(id_cols + ["Month"], as_index=False).agg(agg_dict)

def compute_expected_days(row):
    location = row["Location"]
    weeks = row["Week_Count"]
    return weeks * 1 if location in ["Chennai", "Gurugram"] else weeks * 2

monthly["Expected Workstyle Days"] = monthly.apply(compute_expected_days, axis=1)

In [ ]:
def flatten_monthly_data(df, static_cols, dynamic_metrics, month_col="Month"):
    """
    Converts long-format mona
    thly data into wide-format pivoted data.
    
    Parameters:
    - df (pd.DataFrame): Input monthly data
    - static_cols (list): Columns to keep as identifiers (e.g., Employee Id, Name, BF Levels)
    - dynamic_metrics (list): Columns to pivot across months (e.g., Days at Any Office)
    - month_col (str): Column containing month labels (e.g., 'Apr-25')
    
    Returns:
    - pd.DataFrame: Flattened wide-format data
    """
    df = df.copy()
    df[month_col] = df[month_col].astype(str)  # Ensure flat strings
    df = df.reset_index(drop=True)

    # Melt dynamic metrics into long format
    melted = df.melt(
        id_vars=static_cols + [month_col],
        value_vars=dynamic_metrics,
        var_name="Metric",
        value_name="Value"
    )

    # Create new column: Metric + Month
    melted["Metric_Month"] = melted["Metric"] + " " + melted[month_col]

    # Pivot: one row per employee, one column per Metric_Month
    pivot_df = melted.pivot_table(
        index=static_cols,
        columns="Metric_Month",
        values="Value",
        aggfunc="first"
    )

    # Flatten column index
    pivot_df.columns.name = None
    pivot_df = pivot_df.reset_index()

    return pivot_df


static_cols = [
    "Employee Id", "Employee Name", "BF Level 1 Name", "BF Level 2 Name",
    "BF Level 3 Name", "BF Level 4 Name", "BF Level 5 Name",
    "MD Name", "Manager Name"
]

dynamic_metrics = ["Days at Any Office", "Days at Contracted Office"]

flattened_df = flatten_monthly_data(
    df=monthly_df,
    static_cols=static_cols,
    dynamic_metrics=dynamic_metrics,
    month_col="Month"
)

In [ ]:
import pandas as pd

def summarize_weekly_to_monthly(df, date_col="Week Commencing", id_cols=None, sum_cols=None):
    """
    Aggregates weekly attendance data to monthly totals per employee.

    Parameters:
    - df (pd.DataFrame): Input weekly data
    - date_col (str): Column with week start dates
    - id_cols (list): Columns to group by (e.g., Employee Id, MD Name, BF Level)
    - sum_cols (list): Columns to sum (e.g., Days at Any Office, Days at Contracted Office)

    Returns:
    - pd.DataFrame: Monthly summary per employee
    """
    if id_cols is None:
        id_cols = ["Employee Id"]
    if sum_cols is None:
        sum_cols = ["Days at Any Office", "Days at Contracted Office"]

    # Ensure date column is datetime
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

    # Create Month column in 'Apr-25' format
    df["Month"] = df[date_col].dt.strftime("%b-%y")

    # Group and aggregate
    monthly = df.groupby(id_cols + ["Month"], as_index=False)[sum_cols].sum()

    return monthly

monthly_df = summarize_weekly_to_monthly(
    df=weekly_data,
    id_cols=["Employee Id", "MD Name", "BF Level 3 Name"],
    sum_cols=["Days at Any Office", "Days at Contracted Office"]
)

In [ ]:
import pandas as pd
import re

def normalize_id(val):
    try:
        return str(int(str(val).strip())).zfill(8)
    except:
        return ''

def build_pivotable_trend_table(flattened_path, output_path="pivotable_trend_table.xlsx"):
    df = pd.read_excel(flattened_path)
    df.columns = df.columns.str.strip()

    # Normalize column names
    df.rename(columns={col: col.strip() for col in df.columns}, inplace=True)

    # Normalize IDs
    df['Employee Id'] = df['Employee Id'].apply(normalize_id)
    df['Entity Manager Employee Id'] = df['Entity Manager Employee Id'].apply(normalize_id)
    if 'Md Id' in df.columns:
        df['Md Id'] = df['Md Id'].apply(normalize_id)
    else:
        df['Md Id'] = ''

    # Filter only valid MD traced employees
    df_valid = df[df['Md Found?'].astype(str).str.strip().str.lower() == 'yes'].copy()

    # Detect Workstyle columns like "Sep-25 Workstyle Met? (avg Days)"
    workstyle_cols = [col for col in df_valid.columns if re.search(r'Workstyle Met\?\s*\(avg Days\)', col, re.IGNORECASE)]

    # Melt into long format
    melted = df_valid.melt(
        id_vars=['Employee Id', 'Md Id', 'Md Name'] + [f'Bf Level {i} Name' for i in range(1, 6)],
        value_vars=workstyle_cols,
        var_name='Raw Month',
        value_name='Workstyle Met'
    )

    # Extract month and clean values
    melted['Month'] = melted['Raw Month'].str.extract(r'(^\w{3}-\d{2})')[0]
    melted['Workstyle Met'] = melted['Workstyle Met'].astype(str).str.strip().str.lower()

    # Save to Excel
    melted.to_excel(output_path, index=False)
    print(f"📊 Pivotable trend table saved to '{output_path}'. Use this to build your styled pivot.")
    return melted

In [ ]:
import pandas as pd
import re

def normalize_id(val):
    try:
        return str(int(str(val).strip())).zfill(8)
    except:
        return ''

def build_pivotable_trend_data(flattened_path, output_path="pivotable_trend_data.xlsx"):
    df = pd.read_excel(flattened_path)
    df.columns = df.columns.str.strip()

    # Normalize column names
    df.rename(columns={col: col.strip() for col in df.columns}, inplace=True)

    # Normalize IDs
    df['Employee Id'] = df['Employee Id'].apply(normalize_id)
    df['Entity Manager Employee Id'] = df['Entity Manager Employee Id'].apply(normalize_id)
    if 'Md Id' in df.columns:
        df['Md Id'] = df['Md Id'].apply(normalize_id)
    else:
        df['Md Id'] = ''

    # Filter only valid MD traced employees
    df_valid = df[df['Md Found?'].astype(str).str.strip().str.lower() == 'yes'].copy()

    # Detect Workstyle columns like "Sep-25 Workstyle Met? (avg Days)"
    workstyle_cols = [col for col in df_valid.columns if re.search(r'Workstyle Met\?\s*\(avg Days\)', col, re.IGNORECASE)]

    # Melt into long format
    melted = df_valid.melt(
        id_vars=['Employee Id', 'Md Id', 'Md Name'] + [f'Bf Level {i} Name' for i in range(1, 6)],
        value_vars=workstyle_cols,
        var_name='Raw Month',
        value_name='Workstyle Met'
    )

    # Extract month and clean values
    melted['Month'] = melted['Raw Month'].str.extract(r'(^\w{3}-\d{2})')[0]
    melted['Workstyle Met'] = melted['Workstyle Met'].astype(str).str.strip().str.lower()

    # Save to Excel
    melted.to_excel(output_path, index=False)
    print(f"📊 Pivotable trend data saved to '{output_path}'. Use this for Excel pivot tables.")
    return melted
build_pivotable_trend_data("phase2_flattened_with_md_trace.xlsx")

In [ ]:
import pandas as pd
import re

def normalize_id(val):
    try:
        return str(int(str(val).strip())).zfill(8)
    except:
        return ''

def title_case(col):
    return ' '.join(word.capitalize() for word in col.replace('_', ' ').split())

def build_summary_and_trend_from_flattened(flattened_path, output_path="phase3_summary_and_trend.xlsx"):
    df = pd.read_excel(flattened_path)
    df.columns = df.columns.str.strip()

    # Normalize column names
    df.rename(columns={col: title_case(col) for col in df.columns}, inplace=True)

    # Normalize IDs
    df['Employee Id'] = df['Employee Id'].apply(normalize_id)
    df['Entity Manager Employee Id'] = df['Entity Manager Employee Id'].apply(normalize_id)

    # Detect MD ID column
    md_id_col = next((col for col in df.columns if col.strip().lower() == 'md id'), None)
    if md_id_col:
        df[md_id_col] = df[md_id_col].apply(normalize_id)
    else:
        df['Md Id'] = ''

    # 🧩 MD Summary with GCB 3 → GCB 4 breakdown
    valid_df = df[df['Md Found?'] == 'Yes']
    summary = []

    for (md_id, md_name), group in valid_df.groupby([md_id_col or 'Md Id', 'Md Name']):
        gcb3 = group[group['Global Career Band'] == '3']
        gcb4 = group[group['Global Career Band'] == '4']
        gcb4_under_gcb3 = gcb4[gcb4['Entity Manager Employee Id'].isin(gcb3['Employee Id'])]

        summary.append({
            'MD ID': md_id,
            'MD Name': md_name,
            'GCB 3 Count': len(gcb3),
            'GCB 4 Count': len(gcb4),
            'GCB 4 under GCB 3': len(gcb4_under_gcb3),
            'Total Employees': len(group)
        })

    summary_df = pd.DataFrame(summary)

    # 🧩 Monthly Workstyle Trend
    workstyle_cols = [col for col in df.columns if re.search(r'Workstyle Met\?\s*\(avg Days\)', col, re.IGNORECASE)]
    melted = df.melt(
        id_vars=['Employee Id', 'Md Name'] + [f'Bf Level {i} Name' for i in range(1, 6)],
        value_vars=workstyle_cols,
        var_name='Raw Month',
        value_name='Workstyle Met'
    )

    # Extract month from column names like "Sep-25 Workstyle Met? (avg Days)"
    melted['Month'] = melted['Raw Month'].str.extract(r'(^\w{3}-\d{2})')[0]
    melted['Workstyle Met'] = pd.to_numeric(melted['Workstyle Met'], errors='coerce')

    trend = melted.groupby('Month').agg(
        Headcount=('Employee Id', 'nunique'),
        Workstyle_Met=('Workstyle Met', lambda x: x.notna().sum())
    ).reset_index()

    trend['% Met'] = (trend['Workstyle_Met'] / trend['Headcount'] * 100).round(2)
    trend['MoM Change %'] = trend['% Met'].diff().round(2)

    # 📝 Save both to one sheet
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        workbook = writer.book
        worksheet = workbook.add_worksheet('Summary & Trend')
        writer.sheets['Summary & Trend'] = worksheet

        # Write MD Summary
        summary_df.to_excel(writer, sheet_name='Summary & Trend', startrow=1, startcol=0, index=False)
        worksheet.write(0, 0, 'MD Summary')

        # Write Monthly Trend below
        startrow = len(summary_df) + 4
        trend.to_excel(writer, sheet_name='Summary & Trend', startrow=startrow + 1, startcol=0, index=False)
        worksheet.write(startrow, 0, 'Monthly Workstyle Trend')

    print(f"✅ Summary and trend saved to '{output_path}'.")
    return summary_df, trend
build_summary_and_trend_from_flattened("phase2_flattened_with_md_trace.xlsx")

In [ ]:
import pandas as pd
import re

def normalize_id(val):
    try:
        return str(int(str(val).strip())).zfill(8)
    except:
        return ''

def title_case(col):
    return ' '.join(word.capitalize() for word in col.replace('_', ' ').split())

def build_summary_and_trend_from_flattened(flattened_path, output_path="phase3_summary_and_trend.xlsx"):
    df = pd.read_excel(flattened_path)
    df.columns = df.columns.str.strip()

    # Normalize column names
    df.rename(columns={col: title_case(col) for col in df.columns}, inplace=True)

    # Normalize IDs
    df['Employee Id'] = df['Employee Id'].apply(normalize_id)
    df['Entity Manager Employee Id'] = df['Entity Manager Employee Id'].apply(normalize_id)
    if 'Md Id' in df.columns:
        df['Md Id'] = df['Md Id'].apply(normalize_id)
    else:
        df['Md Id'] = ''

    # 🧩 MD Summary with GCB 3 → GCB 4 breakdown
    valid_df = df[df['Md Found?'] == 'Yes']
    summary = []

    for (md_id, md_name), group in valid_df.groupby(['Md Id', 'Md Name']):
        gcb3 = group[group['Global Career Band'].str.upper() == 'GCB 3']
        gcb4 = group[group['Global Career Band'].str.upper() == 'GCB 4']

        gcb3_count = len(gcb3)
        gcb4_count = len(gcb4)

        # GCB 4 under GCB 3
        gcb4_under_gcb3 = gcb4[gcb4['Entity Manager Employee Id'].isin(gcb3['Employee Id'])]

        summary.append({
            'MD ID': md_id,
            'MD Name': md_name,
            'GCB 3 Count': gcb3_count,
            'GCB 4 Count': gcb4_count,
            'GCB 4 under GCB 3': len(gcb4_under_gcb3),
            'Total Employees': len(group)
        })

    summary_df = pd.DataFrame(summary)

    # 🧩 Monthly Workstyle Trend
    workstyle_cols = [col for col in df.columns if re.match(r'Workstyle Met - ', col)]
    melted = df.melt(
        id_vars=['Employee Id', 'Bf Level', 'Md Id', 'Md Name'] + [f'Bf Level {i} Name' for i in range(1, 6)],
        value_vars=workstyle_cols,
        var_name='Month',
        value_name='Workstyle Met'
    )

    melted['Month'] = melted['Month'].str.replace('Workstyle Met - ', '').str.strip()
    melted['Workstyle Met'] = melted['Workstyle Met'].astype(str).str.strip().str.lower()

    trend = melted.groupby('Month').agg(
        Headcount=('Employee Id', 'nunique'),
        Workstyle_Met=('Workstyle Met', lambda x: (x == 'yes').sum())
    ).reset_index()

    trend['% Met'] = (trend['Workstyle_Met'] / trend['Headcount'] * 100).round(2)
    trend['MoM Change %'] = trend['% Met'].diff().round(2)

    # 📝 Save both to one sheet
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        workbook = writer.book
        worksheet = workbook.add_worksheet('Summary & Trend')
        writer.sheets['Summary & Trend'] = worksheet

        # Write MD Summary
        summary_df.to_excel(writer, sheet_name='Summary & Trend', startrow=1, startcol=0, index=False)
        worksheet.write(0, 0, 'MD Summary')

        # Write Monthly Trend below
        startrow = len(summary_df) + 4
        trend.to_excel(writer, sheet_name='Summary & Trend', startrow=startrow + 1, startcol=0, index=False)
        worksheet.write(startrow, 0, 'Monthly Workstyle Trend')

    print(f"✅ Summary and trend saved to '{output_path}'.")
    return summary_df, trend

In [ ]:
def normalize_id(val):
    try:
        return str(int(str(val).strip())).zfill(8)
    except:
        return ''
    
flat_df['employee id'] = flat_df['employee id'].apply(normalize_id)
flat_df['entity manager employee id'] = flat_df['entity manager employee id'].apply(normalize_id)
gha_df['employee id'] = gha_df['employee id'].apply(normalize_id)
gha_df['entity manager employee id'] = gha_df['entity manager employee id'].apply(normalize_id)

In [ ]:
import pandas as pd
from collections import defaultdict

def title_case(col):
    return ' '.join(word.capitalize() for word in col.replace('_', ' ').split())

def clean_id(val):
    return str(val).strip().replace('\xa0', '').replace('\u200b', '').replace('\r', '').replace('\n', '')

def append_md_info_with_trace(flattened_file_path, gha_file_path,
                              flattened_sheet="flattened",
                              gha_sheet="Headcount Employee - Detail"):
    # Load files
    flat_df = pd.read_excel(flattened_file_path, sheet_name=flattened_sheet)
    gha_df = pd.read_excel(gha_file_path, sheet_name=gha_sheet)

    # Normalize columns
    flat_df.columns = flat_df.columns.str.strip().str.lower()
    gha_df.columns = gha_df.columns.str.strip().str.lower()

    # Clean and normalize IDs
    for df in [flat_df, gha_df]:
        df['employee id'] = df['employee id'].apply(clean_id)
        df['entity manager employee id'] = df['entity manager employee id'].apply(clean_id)

    gha_df['global career band'] = gha_df['global career band'].fillna('').astype(str).str.upper().str.strip()

    # Build multi-record GHA lookup
    gha_multi_lookup = defaultdict(list)
    for _, row in gha_df.iterrows():
        emp_id = row['employee id']
        gha_multi_lookup[emp_id].append(row.to_dict())

    # Prepare output columns
    flat_df['md id'] = ''
    flat_df['md name'] = ''
    flat_df['md found?'] = 'No'
    flat_df['md trace comment'] = ''

    # Traverse each employee
    for idx, row in flat_df.iterrows():
        emp_id = clean_id(row['employee id'])
        current_mgr_id = clean_id(row['entity manager employee id'])
        visited = set()

        while current_mgr_id and current_mgr_id not in visited:
            visited.add(current_mgr_id)
            mgr_records = gha_multi_lookup.get(current_mgr_id, [])

            if not mgr_records:
                flat_df.at[idx, 'md trace comment'] = f"Entity Manager ID missing in GHA: {current_mgr_id}"
                break

            # Try to find MD among duplicates
            md_record = next((r for r in mgr_records if str(r.get('global career band', '')).strip().upper() == 'MD'), None)

            if md_record:
                flat_df.at[idx, 'md id'] = current_mgr_id
                flat_df.at[idx, 'md name'] = md_record.get('employee name', '')
                flat_df.at[idx, 'md found?'] = 'Yes'
                break

            # Continue upward using first available manager
            current_mgr_id = clean_id(mgr_records[0].get('entity manager employee id', ''))

    # Restore column names
    flat_df.columns = [title_case(col) for col in flat_df.columns]

    # Save output
    flat_df.to_excel("phase2_flattened_with_md_trace.xlsx", index=False)
    print("✅ MD tracing complete. Output saved as 'phase2_flattened_with_md_trace.xlsx'.")
    return flat_df

append_md_info_with_trace("phase1_enriched_monthly.xlsx", "Input/gha_sep15.xlsx")

In [ ]:
# updaTED ALL PHASES
import pandas as pd

def enrich_gha_with_manager_gcb(gha_file_path, sheet_name="Headcount Employee - Detail"):
    # Load GHA data
    df = pd.read_excel(gha_file_path, sheet_name=sheet_name)
    df.columns = df.columns.str.strip().str.lower()

    # Normalize IDs
    df['employee id'] = df['employee id'].astype(str).str.strip()
    df['entity manager employee id'] = df['entity manager employee id'].astype(str).str.strip()

    # Build lookup for manager GCB
    gcb_lookup = df.set_index('employee id')['global career band'].to_dict()

    # Map Entity Manager GCB
    df['entity manager gcb'] = df['entity manager employee id'].map(gcb_lookup)

    # Optional: Normalize GCBs
    df['global career band'] = df['global career band'].fillna('').str.upper().str.strip()
    df['entity manager gcb'] = df['entity manager gcb'].fillna('').str.upper().str.strip()

    # Save enriched GHA
    df.to_excel("gha_enriched_with_mgr_gcb.xlsx", index=False)
    print("✅ Phase 1 complete: Entity Manager GCB added.")
    return df
enrich_gha_with_manager_gcb("Input/gha_sep15.xlsx")

In [ ]:
# emp row duplicating due to manager change over the months
import pandas as pd

def get_reporting_tree(manager_id, df_lookup):
    reports = set()
    for emp_id, record in df_lookup.items():
        if record.get('entity manager employee id') == manager_id:
            reports.add(emp_id)
            reports.update(get_reporting_tree(emp_id, df_lookup))
    return reports

def phase3_analysis(flattened_file_path):
    # Load flattened data
    df = pd.read_excel(flattened_file_path, sheet_name="flattened")
    df.columns = df.columns.str.strip().str.lower()

    # Identify monthly columns
    monthly_cols = [col for col in df.columns if any(x in col for x in ['workstyle met?', 'avg days'])]

    # Static columns to preserve
    static_cols = ['employee id', 'employee name', 'global career band', 'md id', 'md name', 'md found?', 'entity manager employee id']

    # Normalize monthly columns
    df[monthly_cols] = df[monthly_cols].fillna('').astype(str).apply(lambda x: x.str.strip().str.lower())

    # Log duplicates
    dupes = df['employee id'].value_counts()
    multi_row_employees = dupes[dupes > 1].index.tolist()
    print(f"🔍 Employees with multiple rows due to manager changes: {len(multi_row_employees)}")

    # Save log
    pd.DataFrame({'employee id': multi_row_employees}).to_excel("phase3_multi_row_log.xlsx", index=False)

    # Merge rows per employee
    grouped = df.groupby('employee id')
    merged_rows = []
    for emp_id, group in grouped:
        merged = {'employee id': emp_id}
        for col in static_cols:
            if col in group.columns:
                merged[col] = group[col].dropna().iloc[-1] if not group[col].dropna().empty else ''
        for col in monthly_cols:
            values = group[col].replace('', pd.NA).dropna()
            merged[col] = values.iloc[0] if not values.empty else ''
        merged_rows.append(merged)

    df_merged = pd.DataFrame(merged_rows)

    # Build lookup
    df_lookup = df_merged.set_index('employee id').to_dict('index')

    # 1️⃣ Hierarchy Summary
    df_merged['global career band'] = df_merged['global career band'].fillna('').str.upper()
    df_merged['md name'] = df_merged['md name'].fillna('').str.strip()

    hierarchy_summary = []
    for md in df_merged['md name'].dropna().unique():
        md_df = df_merged[df_merged['md name'] == md]
        gcb3_df = md_df[md_df['global career band'] == 'GCB 3']
        for _, gcb3 in gcb3_df.iterrows():
            gcb3_id = gcb3['employee id']
            gcb3_name = gcb3['employee name']
            full_subtree = get_reporting_tree(gcb3_id, df_lookup)
            hierarchy_summary.append({
                'MD Name': md,
                'GCB3 ID': gcb3_id,
                'GCB3 Name': gcb3_name,
                'Total Reports Under GCB3': len(full_subtree)
            })
    hierarchy_df = pd.DataFrame(hierarchy_summary)

    # 2️⃣ Monthly Trend Summary
    trend_data = []
    for col in monthly_cols:
        if 'workstyle met?' not in col:
            continue
        month = col.split()[0]
        avg_col = f"{month} avg days at any office per week (with shrinkage)".lower().replace(' ', '_')
        if avg_col not in df_merged.columns:
            continue
        headcount = df_merged[avg_col].notna().sum()
        met_count = df_merged[col].str.strip().str.lower().eq('yes').sum()
        pct = round((met_count / headcount) * 100, 2) if headcount else 0
        trend_data.append({
            'Month': month,
            'Headcount': headcount,
            'Workstyle Met': met_count,
            'Percentage': pct
        })

    trend_df = pd.DataFrame(trend_data)
    trend_df['Month_dt'] = pd.to_datetime(trend_df['Month'], format='%b-%y')
    trend_df = trend_df.sort_values('Month_dt')
    trend_df['MoM Change (%)'] = trend_df['Percentage'].diff().round(2)

    # Save outputs
    with pd.ExcelWriter("phase3_trend_analysis.xlsx", engine='openpyxl') as writer:
        hierarchy_df.to_excel(writer, sheet_name="Hierarchy Summary", index=False)
        trend_df.drop(columns='Month_dt').to_excel(writer, sheet_name="Monthly Trend", index=False)

    print("✅ Phase 3 summary complete. Use draw_phase3_graph(trend_df) to visualize.")
    return trend_df

import matplotlib.pyplot as plt

def draw_phase3_graph(trend_df):
    plt.figure(figsize=(10, 5))
    plt.plot(trend_df['Month_dt'], trend_df['Percentage'], marker='o', color='blue')
    plt.title('Workstyle Met % Trend Over Time')
    plt.xlabel('Month')
    plt.ylabel('% Workstyle Met')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("workstyle_met_trend.png")
    plt.show()
    print("📈 Graph saved as 'workstyle_met_trend.png'.")
    
from phase3_analysis import phase3_analysis
from draw_phase3_graph import draw_phase3_graph

trend_df = phase3_analysis("phase2_flattened_with_md.xlsx")
draw_phase3_graph(trend_df)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def get_reporting_tree(manager_id, df_lookup):
    reports = set()
    for emp_id, record in df_lookup.items():
        if record.get('entity manager employee id') == manager_id:
            reports.add(emp_id)
            reports.update(get_reporting_tree(emp_id, df_lookup))
    return reports

def phase3_analysis(flattened_file_path):
    # Load flattened data
    df = pd.read_excel(flattened_file_path, sheet_name="flattened")
    df.columns = df.columns.str.strip().str.lower()

    # Normalize key columns
    df['employee id'] = df['employee id'].astype(str).str.strip()
    df['entity manager employee id'] = df['entity manager employee id'].astype(str).str.strip()
    df['md name'] = df['md name'].fillna('').str.strip()
    df['global career band'] = df['global career band'].fillna('').str.upper()

    df_lookup = df.set_index('employee id').to_dict('index')

    # 1️⃣ Hierarchy Summary: MD → GCB3 → full subtree count
    hierarchy_summary = []
    for md in df['md name'].dropna().unique():
        md_df = df[df['md name'] == md]
        gcb3_df = md_df[md_df['global career band'] == 'GCB 3']
        for _, gcb3 in gcb3_df.iterrows():
            gcb3_id = gcb3['employee id']
            gcb3_name = gcb3['employee name']
            full_subtree = get_reporting_tree(gcb3_id, df_lookup)
            hierarchy_summary.append({
                'MD Name': md,
                'GCB3 ID': gcb3_id,
                'GCB3 Name': gcb3_name,
                'Total Reports Under GCB3': len(full_subtree)
            })
    hierarchy_df = pd.DataFrame(hierarchy_summary)

    # 2️⃣ Monthly Headcount & Workstyle Met
    month_cols = [col for col in df.columns if 'workstyle met?' in col]
    trend_data = []
    for col in month_cols:
        month = col.split()[0]  # e.g. 'Apr-25'
        avg_col = f"{month} avg days at any office per week (with shrinkage)".lower().replace(' ', '_')
        if avg_col not in df.columns:
            continue
        headcount = df[avg_col].notna().sum()
        met_count = df[col].str.strip().str.lower().eq('yes').sum()
        pct = round((met_count / headcount) * 100, 2) if headcount else 0
        trend_data.append({
            'Month': month,
            'Headcount': headcount,
            'Workstyle Met': met_count,
            'Percentage': pct
        })
    trend_df = pd.DataFrame(trend_data)
    trend_df['Month_dt'] = pd.to_datetime(trend_df['Month'], format='%b-%y')
    trend_df = trend_df.sort_values('Month_dt')
    trend_df['MoM Change (%)'] = trend_df['Percentage'].diff().round(2)

    # 3️⃣ Plot Trend
    plt.figure(figsize=(10, 5))
    plt.plot(trend_df['Month_dt'], trend_df['Percentage'], marker='o', color='blue')
    plt.title('Workstyle Met % Trend Over Time')
    plt.xlabel('Month')
    plt.ylabel('% Workstyle Met')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("workstyle_met_trend.png")
    plt.show()

    # Save outputs
    with pd.ExcelWriter("phase3_trend_analysis.xlsx", engine='openpyxl') as writer:
        hierarchy_df.to_excel(writer, sheet_name="Hierarchy Summary", index=False)
        trend_df.drop(columns='Month_dt').to_excel(writer, sheet_name="Monthly Trend", index=False)

    print("✅ Phase 3 complete. Graph saved as 'workstyle_met_trend.png'.")
    
# phase3_analysis("phase2_flattened_with_md.xlsx", "Input/gha_sep15.xlsx")
phase3_analysis("phase2_flattened_with_md.xlsx")

In [ ]:
# standalone function to find md in gha
import pandas as pd

def trace_md_from_gha(gha_file_path, sheet_name="Headcount Employee - Detail"):
    # Load GHA data
    df = pd.read_excel(gha_file_path, sheet_name=sheet_name)
    df.columns = df.columns.str.strip().str.lower()

    # Normalize IDs
    df['employee id'] = df['employee id'].astype(str).str.strip()
    df['entity manager employee id'] = df['entity manager employee id'].astype(str).str.strip()

    # Select required columns
    required_cols = [
        'employee id', 'employee name', 'global career band',
        'employee business email address', 'legal entity',
        'entity manager employee name', 'functional manager employee name',
        'employee category', 'bf level1 name', 'bf level2 name',
        'bf level3 name', 'bf level4 name', 'bf level5 name',
        'entity manager employee id'
    ]
    df = df[required_cols].copy()

    # Build lookup dictionary
    gha_lookup = df.set_index('employee id').to_dict('index')

    # Identify all MDs
    md_ids = set(df[df['global career band'].str.upper().str.strip() == 'MD']['employee id'])

    # Prepare output columns
    df['md id'] = ''
    df['md name'] = ''
    df['md found?'] = 'No'

    missing_records = []

    for idx, row in df.iterrows():
        current_mgr_id = row.get('entity manager employee id')
        visited = set()
        last_known_mgr_id = None
        last_known_mgr_name = None
        md_found = False

        while pd.notna(current_mgr_id) and current_mgr_id not in visited:
            visited.add(current_mgr_id)
            mgr_record = gha_lookup.get(current_mgr_id)

            if mgr_record is None:
                break

            last_known_mgr_id = current_mgr_id
            last_known_mgr_name = mgr_record.get('employee name', '')

            if current_mgr_id in md_ids:
                df.at[idx, 'md id'] = current_mgr_id
                df.at[idx, 'md name'] = last_known_mgr_name
                df.at[idx, 'md found?'] = 'Yes'
                md_found = True
                break

            current_mgr_id = mgr_record.get('entity manager employee id')

        if not md_found:
            missing_row = row.to_dict()
            missing_row.update({
                'last reachable manager id': last_known_mgr_id,
                'last reachable manager name': last_known_mgr_name,
                'missing reason': 'No MD found in chain'
            })
            missing_records.append(missing_row)

    # Return both DataFrames
    df_with_md = df[df['md found?'] == 'Yes'].copy()
    df_missing_md = pd.DataFrame(missing_records)

    return df_with_md, df_missing_md
df_md, df_missing = trace_md_from_gha("Input/gha_sep15.xlsx")

# Save to Excel
with pd.ExcelWriter("gha_md_trace.xlsx", engine='openpyxl') as writer:
    df_md.to_excel(writer, sheet_name="md found", index=False)
    df_missing.to_excel(writer, sheet_name="md missing", index=False)
    
def neutralize_column_names(df):
    def title_case(col):
        # Replace underscores with spaces, split into words, capitalize each
        return ' '.join(word.capitalize() for word in col.replace('_', ' ').split())

    df.columns = [title_case(col) for col in df.columns]
    return df

# After loading or transforming your DataFrame
df = neutralize_column_names(df)

# Then save or display
df.to_excel("cleaned_output.xlsx", index=False)

In [ ]:
import pandas as pd

def trace_md_and_flag_missing(flattened_file_path, gha_file_path):
    # Load flattened data
    flat_df = pd.read_excel(flattened_file_path, sheet_name="flattened")
    flat_df.columns = flat_df.columns.str.strip().str.lower()

    # Load GHA data
    gha_df = pd.read_excel(gha_file_path, sheet_name="Headcount Employee - Detail")
    gha_df.columns = gha_df.columns.str.strip().str.lower()
    gha_df = gha_df.drop_duplicates(subset='employee id', keep='last')

    # Normalize IDs to string
    gha_df['employee id'] = gha_df['employee id'].astype(str).str.strip()
    gha_df['entity manager employee id'] = gha_df['entity manager employee id'].astype(str).str.strip()
    flat_df['employee id'] = flat_df['employee id'].astype(str).str.strip()
    flat_df['entity manager employee id'] = flat_df['entity manager employee id'].astype(str).str.strip()

    # Build lookup dictionary
    gha_lookup = gha_df.set_index('employee id').to_dict('index')

    # Identify all MDs
    md_df = gha_df[gha_df['global career band'].str.strip().str.upper() == 'MD']
    md_ids = set(md_df['employee id'])

    # Prepare output columns
    flat_df['md id'] = ''
    flat_df['md name'] = ''
    flat_df['md found?'] = 'No'

    missing_records = []

    for idx, row in flat_df.iterrows():
        current_mgr_id = row.get('entity manager employee id')
        visited = set()
        last_known_mgr_id = None
        last_known_mgr_name = None
        md_found = False

        while pd.notna(current_mgr_id) and current_mgr_id not in visited:
            visited.add(current_mgr_id)
            mgr_record = gha_lookup.get(current_mgr_id)

            if mgr_record is None:
                break

            last_known_mgr_id = current_mgr_id
            last_known_mgr_name = mgr_record.get('employee name', '')

            if current_mgr_id in md_ids:
                flat_df.at[idx, 'md id'] = current_mgr_id
                flat_df.at[idx, 'md name'] = last_known_mgr_name
                flat_df.at[idx, 'md found?'] = 'Yes'
                md_found = True
                break

            current_mgr_id = mgr_record.get('entity manager employee id')

        if not md_found:
            missing_row = row.to_dict()
            missing_row.update({
                'last reachable manager id': last_known_mgr_id,
                'last reachable manager name': last_known_mgr_name,
                'missing reason': 'No MD found in chain'
            })
            missing_records.append(missing_row)

    # Save both sheets
    with pd.ExcelWriter("phase2_flattened_with_md.xlsx", engine='openpyxl') as writer:
        flat_df.to_excel(writer, sheet_name="flattened", index=False)
        pd.DataFrame(missing_records).to_excel(writer, sheet_name="missing md chain", index=False)

    print(f"✅ MD tracing complete: {flat_df['md found?'].value_counts().to_dict()}")
trace_md_and_flag_missing("phase2_flattened.xlsx", "Input/gha_sep15.xlsx")

In [ ]:
## Fine for MD finding
import pandas as pd

def append_md_info(flattened_file_path, gha_file_path):
    # Load flattened data
    flat_df = pd.read_excel(flattened_file_path, sheet_name="flattened")
    flat_df.columns = flat_df.columns.str.strip().str.lower()

    # Load GHA data
    gha_df = pd.read_excel(gha_file_path, sheet_name="Headcount Employee - Detail")
    gha_df.columns = gha_df.columns.str.strip().str.lower()
    gha_df = gha_df.drop_duplicates(subset='employee id', keep='last')

    # Build lookup dictionary
    gha_lookup = gha_df.set_index('employee id').to_dict('index')

    # Identify all MDs
    md_df = gha_df[gha_df['global career band'].str.strip().str.upper() == 'MD']
    md_ids = set(md_df['employee id'])

    # Prepare output columns
    flat_df['md id'] = ''
    flat_df['md name'] = ''
    flat_df['md found?'] = 'No'

    for idx, row in flat_df.iterrows():
        current_mgr_id = row.get('entity manager employee id')
        visited = set()

        while pd.notna(current_mgr_id) and current_mgr_id not in visited:
            visited.add(current_mgr_id)
            mgr_record = gha_lookup.get(current_mgr_id)

            if mgr_record is None:
                break

            if current_mgr_id in md_ids:
                flat_df.at[idx, 'md id'] = current_mgr_id
                flat_df.at[idx, 'md name'] = mgr_record.get('employee name', '')
                flat_df.at[idx, 'md found?'] = 'Yes'
                break

            current_mgr_id = mgr_record.get('entity manager employee id')

    # Save updated file
    flat_df.to_excel("phase2_flattened_with_md.xlsx", sheet_name="flattened", index=False)
    print(f"✅ MD tracing complete: {flat_df['md found?'].value_counts().to_dict()}")
    
append_md_info("phase2_flattened.xlsx", "Input/gha_sep15.xlsx")

In [ ]:
# perfect
import pandas as pd

def flatten_monthly_data(enriched_file_path):
    # Load enriched data
    df = pd.read_excel(enriched_file_path, sheet_name="enriched_data")
    df.columns = df.columns.str.strip().str.lower()

    # Normalize month column
    df['month'] = df['month'].str.strip().str.title()  # e.g. 'Feb-25'
    df['month_dt'] = pd.to_datetime(df['month'], format='%b-%y')

    # Static columns
    static_cols = [
        'employee id', 'employee name', 'entity manager employee id',
        'entity manager employee name', 'global career band'
    ]

    # Dynamic metrics
    dynamic_metrics = [
        'avg days at any office per week (with shrinkage)',
        'workstyle met?'
    ]

    # Pivot
    pivot_df = df.pivot_table(
        index=static_cols,
        columns='month',
        values=dynamic_metrics,
        aggfunc='first'
    )

    # Sort months chronologically
    sorted_months = (
        df[['month', 'month_dt']]
        .drop_duplicates()
        .sort_values('month_dt')['month']
        .tolist()
    )

    # Build ordered column list: [Feb-25 avg, Feb-25 workstyle, Mar-25 avg, …]
    ordered_cols = []
    for month in sorted_months:
        for metric in dynamic_metrics:
            col_key = (metric.lower(), month)
            if col_key in pivot_df.columns:
                ordered_cols.append(col_key)

    # Reindex and flatten
    pivot_df = pivot_df[ordered_cols]
    pivot_df.columns = [
        f"{month} {metric.replace(' ', '_')}"
        for metric, month in pivot_df.columns
    ]
    pivot_df.reset_index(inplace=True)

    # Save to Excel
    pivot_df.to_excel("phase2_flattened.xlsx", sheet_name="flattened", index=False)
    print(f"✅ Flattened file created with grouped monthly columns.")

In [ ]:
import pandas as pd

def build_hierarchy_to_muthu(enriched_file_path, gha_file_path):
    # Load enriched data
    enriched_df = pd.read_excel(enriched_file_path, sheet_name="enriched_data")
    enriched_df.columns = enriched_df.columns.str.strip().str.lower()

    # Load GHA data
    gha_df = pd.read_excel(gha_file_path, sheet_name="Headcount Employee - Detail")
    gha_df.columns = gha_df.columns.str.strip().str.lower()

    # Deduplicate GHA
    gha_df = gha_df.drop_duplicates(subset='employee id', keep='last')

    # Build lookup dictionary and valid ID set
    gha_lookup = gha_df.set_index('employee id').to_dict('index')
    valid_mgr_ids = set(gha_df['employee id'])

    # Identify Muthu's employee ID
    muthu_df = gha_df[
        (gha_df['global career band'].str.strip().str.upper() == 'MD') &
        (gha_df['employee name'].str.strip().str.lower() == 'muthu')
    ]
    if muthu_df.empty:
        print("❗ Muthu not found in GHA file.")
        return
    muthu_id = muthu_df['employee id'].iloc[0]
    muthu_name = muthu_df['employee name'].iloc[0]

    hierarchy_records = []
    missing_chain_records = []

    for _, row in enriched_df.iterrows():
        emp_id = row['employee id']
        current_mgr_id = row.get('entity manager employee id')
        visited = set()
        reached_muthu = False
        missing_mgr_id = None
        missing_mgr_name = None

        # Capture immediate manager info from enriched row
        immediate_mgr_id = current_mgr_id
        immediate_mgr_name = row.get('entity manager employee name')
        immediate_mgr_gcb = gha_lookup.get(immediate_mgr_id, {}).get('global career band', '')

        while pd.notna(current_mgr_id) and current_mgr_id not in visited:
            visited.add(current_mgr_id)

            if current_mgr_id not in valid_mgr_ids:
                # First missing manager in chain
                missing_mgr_id = current_mgr_id
                missing_mgr_name = (
                    immediate_mgr_name if current_mgr_id == immediate_mgr_id else ''
                )
                break

            if current_mgr_id == muthu_id:
                reached_muthu = True
                break

            current_mgr_id = gha_lookup[current_mgr_id].get('entity manager employee id')

        if reached_muthu:
            enriched_row = row.to_dict()
            enriched_row.update({
                'md id': muthu_id,
                'md name': muthu_name,
                'entity manager employee id': immediate_mgr_id,
                'entity manager employee name': immediate_mgr_name,
                'entity manager gcb': immediate_mgr_gcb
            })
            hierarchy_records.append(enriched_row)
        elif missing_mgr_id:
            missing_row = row.to_dict()
            missing_row.update({
                'missing manager id': missing_mgr_id,
                'missing manager name': missing_mgr_name,
                'missing reason': 'Manager not found in GHA'
            })
            missing_chain_records.append(missing_row)

    # Save to Excel
    hierarchy_df = pd.DataFrame(hierarchy_records)
    missing_df = pd.DataFrame(missing_chain_records)

    with pd.ExcelWriter("phase2_hierarchy.xlsx", engine='openpyxl') as writer:
        hierarchy_df.to_excel(writer, sheet_name="hierarchy report", index=False)
        missing_df.to_excel(writer, sheet_name="missing mgr in gha", index=False)

    print("✅ Phase 2 complete: 'phase2_hierarchy.xlsx' created.")

In [ ]:
import pandas as pd

def enrich_monthly_data(monthly_csv_path, gha_excel_path):
    # Load monthly tracker CSV
    monthly_df = pd.read_csv(monthly_csv_path)
    
    # Load GHA master Excel sheet
    gha_df = pd.read_excel(gha_excel_path, sheet_name="Headcount Employee - Detail")

    # Standardize column names
    monthly_df.columns = monthly_df.columns.str.strip().str.lower()
    gha_df.columns = gha_df.columns.str.strip().str.lower()

    # Deduplicate GHA data by Employee ID (keep last occurrence)
    gha_df = gha_df.drop_duplicates(subset='employee id', keep='last')

    # Define columns to bring from GHA
    gha_columns = [
        'employee id', 'global career band',
        'bf level 1 name', 'bf level 2 name', 'bf level 3 name',
        'bf level 4 name', 'bf level 5 name',
        'legal entity name', 'entity manager employee id', 'entity manager employee name'
    ]

    # Filter GHA to required columns
    gha_filtered = gha_df[gha_columns]

    # Merge monthly data with GHA data
    enriched_df = monthly_df.merge(
        gha_filtered,
        on='employee id',
        how='left',
        suffixes=('', '_gha')
    )

    # Identify rows where enrichment failed (missing GHA info)
    missing_mask = enriched_df['global career band'].isna()
    missing_df = enriched_df[missing_mask].copy()
    enriched_final_df = enriched_df.copy()  # Keep all rows, enriched where possible

    # Save to Excel with two sheets
    with pd.ExcelWriter("phase1_enriched.xlsx", engine='openpyxl') as writer:
        enriched_final_df.to_excel(writer, sheet_name="enriched_data", index=False)
        missing_df.to_excel(writer, sheet_name="missing in gha", index=False)

    print("✅ Phase 1 complete: 'phase1_enriched.xlsx' created with enriched and missing sheets.")
enrich_monthly_data("Input/monthlysep.csv", "Input/gha_sep15.xlsx")

In [ ]:
import pandas as pd

def enrich_employee_data(monthly_csv_path, gha_file_path):
    # Load monthly employee data from CSV
    monthly_df = pd.read_csv(monthly_csv_path)
    gha_df = pd.read_excel(gha_file_path, sheet_name="Headcount – Employee Detail")

    # Standardize column names
    monthly_df.columns = monthly_df.columns.str.strip().str.lower()
    gha_df.columns = gha_df.columns.str.strip().str.lower()

    # Deduplicate GHA data by Employee ID (keep last occurrence)
    gha_df = gha_df.drop_duplicates(subset='employee id', keep='last')

    # Merge GHA info into monthly data
    enriched_df = monthly_df.merge(
        gha_df[['employee id', 'employee name', 'entity manager employee id', 'global career band', 'bf levels', 'email']],
        on='employee id',
        how='left',
        suffixes=('', '_gha')
    )

    # Identify missing employee info
    missing_employee_df = enriched_df[enriched_df['global career band'].isna()].copy()
    missing_employee_df['missing_reason'] = 'Employee not found in GHA'

    # Check for missing manager info using GHA lookup
    gha_ids = set(gha_df['employee id'])
    enriched_df['manager_missing'] = ~enriched_df['entity manager employee id'].isin(gha_ids)
    missing_manager_df = enriched_df[enriched_df['manager_missing']].copy()
    missing_manager_df['missing_reason'] = 'Manager not found in GHA'

    # Combine all missing info
    missing_combined_df = pd.concat([missing_employee_df, missing_manager_df], ignore_index=True)

    # Save outputs
    with pd.ExcelWriter("phase1_enriched.xlsx", engine='openpyxl') as writer:
        enriched_df.to_excel(writer, sheet_name="Enriched", index=False)
        missing_combined_df.to_excel(writer, sheet_name="Missing Info", index=False)

In [ ]:
import pandas as pd

def generate_phase2_summary_df(monthly_df, hierarchy_df, missing_df):
    summary_data = []

    # 1. Total Employees in Monthly.csv
    total_headcount = len(monthly_df)
    summary_data.append({"Metric": "Total Headcount", "Value": total_headcount})

    # 2. Missing records in GHA
    missing_count = len(missing_df)
    summary_data.append({
        "Metric": "Total Records with Missing Information",
        "Value": missing_count
    })

    # 3. Total in Hierarchy sheet
    total_available = len(hierarchy_df)
    summary_data.append({"Metric": "Total Available Headcount", "Value": total_available})

    # 4–8. Total per BF Level 1–5
    for level in range(1, 6):
        col_name = f"BF Level {level}"
        if col_name in hierarchy_df.columns:
            counts = hierarchy_df[col_name].value_counts()
            for k, v in counts.items():
                summary_data.append({
                    "Metric": f"Total Headcount for {col_name} = {k}",
                    "Value": v
                })

    # 9. Finance except Poland
    if "Work Location Country/Territory Name" in hierarchy_df.columns:
        poland_count = (hierarchy_df["Work Location Country/Territory Name"] == "Poland").sum()
        finance_except_poland = total_available - poland_count
        summary_data.append({
            "Metric": "Total Headcount for Finance except Poland",
            "Value": finance_except_poland
        })

    # 10. MD count
    if "Global Career Band" in hierarchy_df.columns:
        md_count = (hierarchy_df["Global Career Band"] == "MD").sum()
        summary_data.append({"Metric": "Total Number of MDs", "Value": md_count})

        # 11. GCB 3 count
        gcb3_count = (hierarchy_df["Global Career Band"] == "3").sum()
        summary_data.append({"Metric": "Total Number of GCB 3s", "Value": gcb3_count})

    # Final DF
    summary_df = pd.DataFrame(summary_data)
    return summary_df



summary_df = generate_phase2_summary_df(monthly_df, hierarchy_df, missing_df)
with pd.ExcelWriter("phase2_output.xlsx", engine="openpyxl") as writer:
    hierarchy_df.to_excel(writer, sheet_name="Hierarchy Report", index=False)
    missing_df.to_excel(writer, sheet_name="Missing Managers", index=False)
    summary_df.to_excel(writer, sheet_name="Summary", index=False)


In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import os

# -----------------------------
# Recursive slicing function
# -----------------------------
def get_hierarchy_slice(df, manager_id):
    """Return all employees (direct/indirect) under manager_id in order.
       Manager's own record is NOT included.
    """
    result = []
    visited = set()

    mid_str = str(manager_id).strip()

    def recurse(mid):
        reports = df[df['Entity Manager Employee ID'].astype(str).str.strip() == str(mid).strip()]
        for _, r in reports.iterrows():
            eid = str(r['Employee ID']).strip()
            if eid not in visited:
                visited.add(eid)
                result.append(r.to_dict())
                recurse(eid)

    recurse(mid_str)
    return pd.DataFrame(result)


# -----------------------------
# GUI Application
# -----------------------------
class Phase3GUI:
    def __init__(self, master):
        self.master = master
        master.title("Phase 3: Hierarchy Slice Utility")
        master.geometry("700x400")

        self.df = None
        self.name_to_ids = {}
        self.output_folder = ""

        # ---------------- Input File ----------------
        tk.Label(master, text="Hierarchy Report File:").grid(row=0, column=0, sticky="w", padx=10, pady=5)
        self.input_file_var = tk.StringVar()
        tk.Entry(master, textvariable=self.input_file_var, width=50).grid(row=0, column=1, padx=5)
        tk.Button(master, text="Browse", command=self.browse_input_file).grid(row=0, column=2, padx=5)

        # ---------------- Output Folder ----------------
        tk.Label(master, text="Output Folder:").grid(row=1, column=0, sticky="w", padx=10, pady=5)
        self.output_folder_var = tk.StringVar()
        tk.Entry(master, textvariable=self.output_folder_var, width=50).grid(row=1, column=1, padx=5)
        tk.Button(master, text="Browse", command=self.browse_output_folder).grid(row=1, column=2, padx=5)

        # ---------------- Manager Selection ----------------
        tk.Label(master, text="Select Manager Name:").grid(row=2, column=0, sticky="w", padx=10, pady=5)
        self.manager_cb = ttk.Combobox(master, width=47, state="readonly")
        self.manager_cb.grid(row=2, column=1, padx=5)
        tk.Button(master, text="Load Managers", command=self.load_managers).grid(row=2, column=2, padx=5)

        # ---------------- GCB Level Selection ----------------
        tk.Label(master, text="Select GCB Level (for Generate All):").grid(row=3, column=0, sticky="w", padx=10, pady=5)
        self.gcb_level_cb = ttk.Combobox(master, values=[3,4,5,6,7,8], width=10, state="readonly")
        self.gcb_level_cb.grid(row=3, column=1, sticky="w", padx=5)

        # ---------------- Buttons ----------------
        tk.Button(master, text="Generate Slice for Selected Manager", command=self.slice_selected_manager).grid(row=4, column=0, columnspan=2, pady=10)
        tk.Button(master, text="Generate Slices for All at GCB Level", command=self.slice_all_at_level).grid(row=5, column=0, columnspan=2, pady=10)

        # ---------------- Status ----------------
        self.status_var = tk.StringVar()
        tk.Label(master, textvariable=self.status_var, fg="blue").grid(row=6, column=0, columnspan=3, pady=10)


    # ---------------- Browse Functions ----------------
    def browse_input_file(self):
        filename = filedialog.askopenfilename(filetypes=[("Excel files","*.xlsx")])
        if filename:
            self.input_file_var.set(filename)

    def browse_output_folder(self):
        folder = filedialog.askdirectory()
        if folder:
            self.output_folder_var.set(folder)
            self.output_folder = folder

    # ---------------- Load Manager Names ----------------
    def load_managers(self):
        file = self.input_file_var.get()
        if not file or not os.path.exists(file):
            messagebox.showerror("Error", "Please select a valid Hierarchy Report file")
            return

        self.df = pd.read_excel(file, sheet_name="Hierarchy Report")
        self.df.columns = self.df.columns.str.strip()

        # Normalize ID columns
        if 'Employee ID' in self.df.columns:
            self.df['Employee ID'] = self.df['Employee ID'].astype(str).str.strip()
        if 'Entity Manager Employee ID' in self.df.columns:
            self.df['Entity Manager Employee ID'] = self.df['Entity Manager Employee ID'].astype(str).str.strip()
        if 'Global Career Band' in self.df.columns:
            self.df['Global Career Band'] = self.df['Global Career Band'].astype(str).str.strip()

        # ✅ Only names from "Manager Name" column
        managers = sorted(self.df['Manager Name'].dropna().unique().tolist())
        self.manager_cb['values'] = managers

        # Create name→IDs map (still needed internally)
        self.name_to_ids = self.df.groupby('Employee Name')['Employee ID'].apply(lambda s: list(dict.fromkeys(s))).to_dict()
        self.status_var.set(f"Loaded {len(managers)} managers.")


    # ---------------- Slice Selected Manager ----------------
    def slice_selected_manager(self):
        name = self.manager_cb.get()
        out_folder = self.output_folder_var.get()
        if not name:
            messagebox.showerror("Error", "Please select a manager name")
            return
        if not out_folder:
            messagebox.showerror("Error", "Please select output folder")
            return

        ids = self.name_to_ids.get(name, [])
        if len(ids) > 1:
            id_select_win = tk.Toplevel(self.master)
            id_select_win.title(f"Select Manager ID for {name}")
            tk.Label(id_select_win, text=f"Select Manager ID for {name}:").pack(pady=5)
            selected_id_var = tk.StringVar()
            cb = ttk.Combobox(id_select_win, values=ids, textvariable=selected_id_var, state="readonly")
            cb.pack(pady=5)
            def confirm_id():
                manager_id = selected_id_var.get()
                if manager_id:
                    self.generate_slice(manager_id, name, out_folder)
                    id_select_win.destroy()
            tk.Button(id_select_win, text="Confirm", command=confirm_id).pack(pady=5)
            return
        elif len(ids) == 0:
            messagebox.showerror("Error", f"No manager ID found for {name}")
            return
        else:
            manager_id = ids[0]
            self.generate_slice(manager_id, name, out_folder)

    # ---------------- Slice All at GCB Level ----------------
    def slice_all_at_level(self):
        level = self.gcb_level_cb.get()
        out_folder = self.output_folder_var.get()
        if not level:
            messagebox.showerror("Error", "Please select a GCB level")
            return
        if not out_folder:
            messagebox.showerror("Error", "Please select output folder")
            return
        level = int(level)
        gcb_numeric = pd.to_numeric(self.df['Global Career Band'], errors='coerce')
        managers = self.df[gcb_numeric == level][['Employee ID','Employee Name']].drop_duplicates()

        saved = 0
        for _, row in managers.iterrows():
            mid = str(row['Employee ID'])
            mname = row['Employee Name']
            sliced_df = get_hierarchy_slice(self.df, mid)
            if sliced_df.empty:
                continue
            summary = self.build_summary(sliced_df, mname, mid, level)

            file_path = os.path.join(out_folder, f"{mname}_{mid}_slice.xlsx")
            with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
                sliced_df.to_excel(writer, index=False, sheet_name="Hierarchy Slice")
                pd.DataFrame(summary).to_excel(writer, index=False, sheet_name="Summary")
            saved += 1

        messagebox.showinfo("Done", f"All slices for GCB Level {level} saved in:\n{out_folder}\nFiles created: {saved}")

    # ---------------- Generate Slice (single) ----------------
    def generate_slice(self, manager_id, manager_name, out_folder):
        sliced_df = get_hierarchy_slice(self.df, manager_id)
        if sliced_df.empty:
            messagebox.showinfo("Info", f"No records found under {manager_name}")
            return

        gcb_level = self.gcb_level_cb.get()
        gcb_level = int(gcb_level) if gcb_level else None
        summary = self.build_summary(sliced_df, manager_name, manager_id, gcb_level)

        safe_name = manager_name.replace(" ", "_")
        file_path = os.path.join(out_folder, f"{safe_name}_{manager_id}_slice.xlsx")
        with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
            sliced_df.to_excel(writer, index=False, sheet_name="Hierarchy Slice")
            pd.DataFrame(summary).to_excel(writer, index=False, sheet_name="Summary")

        self.status_var.set(f"Slice for {manager_name} saved → {file_path}")
        messagebox.showinfo("Success", f"Slice for {manager_name} saved successfully!")

    # ---------------- Summary helper ----------------
    def build_summary(self, sliced_df, manager_name, manager_id, gcb_level):
        gcb_numeric = pd.to_numeric(sliced_df['Global Career Band'], errors='coerce')

        total_emp = sliced_df['Employee ID'].nunique()
        total_gcb3 = sliced_df[gcb_numeric == 3]['Employee ID'].nunique()
        total_gcb4 = sliced_df[gcb_numeric == 4]['Employee ID'].nunique()

        summary = [
            {"Metric": "Manager Name", "Value": manager_name},
            {"Metric": "Manager ID", "Value": manager_id},
            {"Metric": "Total Employees (direct+indirect)", "Value": total_emp},
        ]

        # ✅ Logic fix: show only relevant counts
        if gcb_level == 3:
            summary.append({"Metric": "Total GCB4 reporting", "Value": total_gcb4})
        elif gcb_level == 4:
            summary.append({"Metric": "Direct Employees under GCB4", "Value": total_emp})

        return summary


# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    root = tk.Tk()
    app = Phase3GUI(root)
    root.mainloop()


In [ ]:
import os
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, ttk


# -------------------------
# Helper functions
# -------------------------

def get_all_reports(df, manager_name):
    """Get all reports under a manager recursively"""
    reports = df[df["Manager Name"] == manager_name].copy()
    all_reports = reports.copy()

    for _, row in reports.iterrows():
        sub_reports = get_all_reports(df, row["Entity Manager Employee name"])
        all_reports = pd.concat([all_reports, sub_reports], ignore_index=True)

    return all_reports


def generate_summary(sliced_df, manager_name, gcb_level):
    """Generate summary dataframe (dummy example)"""
    summary = (
        sliced_df.groupby("Global Career Band")
        .size()
        .reset_index(name="Count")
    )
    summary["Manager"] = manager_name
    summary["Level"] = gcb_level
    return summary


def generate_slice(input_file, output_dir, manager_name, gcb_level, suppress_popup=False):
    df = pd.read_excel(input_file, sheet_name="Hierarchy Report")

    # Managers list
    managers = df["Entity Manager Employee name"].dropna().unique().tolist()
    if manager_name not in managers:
        if not suppress_popup:
            messagebox.showerror("Error", f"{manager_name} is not a valid manager.")
        return

    # Get reports
    sliced_df = get_all_reports(df, manager_name).copy()
    print(f"{manager_name} → {len(sliced_df)} rows")  # debug

    # Remove the manager’s own record
    sliced_df = sliced_df[sliced_df["Entity Manager Employee name"] != manager_name]

    # Summary
    summary_df = generate_summary(sliced_df, manager_name, gcb_level)

    # Save
    out_file = os.path.join(output_dir, f"{manager_name}_slice.xlsx")
    with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
        sliced_df.to_excel(writer, sheet_name="Slice", index=False)
        summary_df.to_excel(writer, sheet_name="Summary", index=False)

    if not suppress_popup:
        messagebox.showinfo("Success", f"Report saved to {out_file}")


def generate_all(input_file, output_dir, gcb_level):
    df = pd.read_excel(input_file, sheet_name="Hierarchy Report")

    managers = df.loc[
        df["Global Career Band"] == gcb_level, "Entity Manager Employee name"
    ].dropna().unique().tolist()

    if not managers:
        messagebox.showwarning("Warning", f"No managers found for {gcb_level}")
        return

    for m in managers:
        generate_slice(input_file, output_dir, m, gcb_level, suppress_popup=True)

    messagebox.showinfo("Success", f"All {gcb_level} slices saved to {output_dir}")


# -------------------------
# GUI
# -------------------------

class ReportGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Phase 3 Hierarchy Slicer")

        # Vars
        self.input_file = tk.StringVar()
        self.output_dir = tk.StringVar()
        self.manager_name = tk.StringVar()
        self.gcb_level = tk.StringVar()

        # Layout
        tk.Label(root, text="Input Excel File:").grid(row=0, column=0, sticky="w")
        tk.Entry(root, textvariable=self.input_file, width=50).grid(row=0, column=1)
        tk.Button(root, text="Browse", command=self.browse_input).grid(row=0, column=2)

        tk.Label(root, text="Output Folder:").grid(row=1, column=0, sticky="w")
        tk.Entry(root, textvariable=self.output_dir, width=50).grid(row=1, column=1)
        tk.Button(root, text="Browse", command=self.browse_output).grid(row=1, column=2)

        tk.Label(root, text="Manager Name:").grid(row=2, column=0, sticky="w")
        self.manager_combo = ttk.Combobox(root, textvariable=self.manager_name, width=47)
        self.manager_combo.grid(row=2, column=1, columnspan=2)

        tk.Label(root, text="GCB Level:").grid(row=3, column=0, sticky="w")
        self.gcb_combo = ttk.Combobox(
            root,
            textvariable=self.gcb_level,
            values=["MD", "GCB3", "GCB4", "GCB5", "GCB6", "GCB7", "GCB8"],
            width=20,
        )
        self.gcb_combo.grid(row=3, column=1, sticky="w")

        # Buttons
        tk.Button(
            root, text="Generate Slice (Selected Manager)", command=self.run_slice
        ).grid(row=4, column=0, pady=10)

        tk.Button(
            root, text="Generate All (Selected GCB Level)", command=self.run_all
        ).grid(row=4, column=1, pady=10)

    def browse_input(self):
        file = filedialog.askopenfilename(
            filetypes=[("Excel Files", "*.xlsx *.xls")])
        if file:
            self.input_file.set(file)
            # Load managers into combobox
            try:
                df = pd.read_excel(file, sheet_name="Hierarchy Report")
                managers = df["Entity Manager Employee name"].dropna().unique().tolist()
                self.manager_combo["values"] = sorted(managers)
            except Exception as e:
                messagebox.showerror("Error", f"Could not load managers: {e}")

    def browse_output(self):
        folder = filedialog.askdirectory()
        if folder:
            self.output_dir.set(folder)

    def run_slice(self):
        if not self.input_file.get() or not self.output_dir.get():
            messagebox.showerror("Error", "Please select input file and output folder")
            return
        if not self.manager_name.get():
            messagebox.showerror("Error", "Please select a manager name")
            return
        if not self.gcb_level.get():
            messagebox.showerror("Error", "Please select a GCB level")
            return

        generate_slice(
            self.input_file.get(),
            self.output_dir.get(),
            self.manager_name.get(),
            self.gcb_level.get(),
        )

    def run_all(self):
        if not self.input_file.get() or not self.output_dir.get():
            messagebox.showerror("Error", "Please select input file and output folder")
            return
        if not self.gcb_level.get():
            messagebox.showerror("Error", "Please select a GCB level")
            return

        generate_all(
            self.input_file.get(),
            self.output_dir.get(),
            self.gcb_level.get(),
        )


# -------------------------
# Run
# -------------------------

if __name__ == "__main__":
    root = tk.Tk()
    app = ReportGUI(root)
    root.mainloop()


In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import os

# -----------------------------
# Recursive slicing function
# -----------------------------
def get_hierarchy_slice(df, manager_id):
    """Return all employees (direct/indirect) under manager_id in order."""
    result = []

    def recurse(mid):
        reports = df[df['Entity Manager Employee ID'] == mid]
        for _, r in reports.iterrows():
            result.append(r)
            recurse(r['Employee ID'])
    
    # Add manager row itself
    manager_row = df[df['Employee ID'] == manager_id]
    if not manager_row.empty:
        result.append(manager_row.iloc[0])
        recurse(manager_id)
    return pd.DataFrame(result)

# -----------------------------
# GUI Application
# -----------------------------
class Phase3GUI:
    def __init__(self, master):
        self.master = master
        master.title("Phase 3: Hierarchy Slice Utility")
        master.geometry("700x400")

        # ---------------- Input File ----------------
        tk.Label(master, text="Hierarchy Report File:").grid(row=0, column=0, sticky="w", padx=10, pady=5)
        self.input_file_var = tk.StringVar()
        tk.Entry(master, textvariable=self.input_file_var, width=50).grid(row=0, column=1, padx=5)
        tk.Button(master, text="Browse", command=self.browse_input_file).grid(row=0, column=2, padx=5)

        # ---------------- Output Folder ----------------
        tk.Label(master, text="Output Folder:").grid(row=1, column=0, sticky="w", padx=10, pady=5)
        self.output_folder_var = tk.StringVar()
        tk.Entry(master, textvariable=self.output_folder_var, width=50).grid(row=1, column=1, padx=5)
        tk.Button(master, text="Browse", command=self.browse_output_folder).grid(row=1, column=2, padx=5)

        # ---------------- Manager Selection ----------------
        tk.Label(master, text="Select Manager Name:").grid(row=2, column=0, sticky="w", padx=10, pady=5)
        self.manager_cb = ttk.Combobox(master, width=47, state="readonly")
        self.manager_cb.grid(row=2, column=1, padx=5)
        tk.Button(master, text="Load Managers", command=self.load_managers).grid(row=2, column=2, padx=5)

        # ---------------- GCB Level Selection ----------------
        tk.Label(master, text="Select GCB Level (for Generate All):").grid(row=3, column=0, sticky="w", padx=10, pady=5)
        self.gcb_level_cb = ttk.Combobox(master, values=[3,4,5,6,7,8], width=10, state="readonly")
        self.gcb_level_cb.grid(row=3, column=1, sticky="w", padx=5)

        # ---------------- Buttons ----------------
        tk.Button(master, text="Generate Slice for Selected Manager", command=self.slice_selected_manager).grid(row=4, column=0, columnspan=2, pady=10)
        tk.Button(master, text="Generate Slices for All at GCB Level", command=self.slice_all_at_level).grid(row=5, column=0, columnspan=2, pady=10)

        # ---------------- Status ----------------
        self.status_var = tk.StringVar()
        tk.Label(master, textvariable=self.status_var, fg="blue").grid(row=6, column=0, columnspan=3, pady=10)

    # ---------------- Browse Functions ----------------
    def browse_input_file(self):
        filename = filedialog.askopenfilename(filetypes=[("Excel files","*.xlsx")])
        if filename:
            self.input_file_var.set(filename)

    def browse_output_folder(self):
        folder = filedialog.askdirectory()
        if folder:
            self.output_folder_var.set(folder)

    # ---------------- Load Manager Names ----------------
    def load_managers(self):
        file = self.input_file_var.get()
        if not file or not os.path.exists(file):
            messagebox.showerror("Error", "Please select a valid Hierarchy Report file")
            return
        self.df = pd.read_excel(file, sheet_name="Hierarchy Report")
        # Create a map: Name -> list of IDs
        self.name_to_ids = self.df.groupby('Employee Name')['Employee ID'].apply(list).to_dict()
        self.manager_cb['values'] = list(self.name_to_ids.keys())
        self.status_var.set(f"Loaded {len(self.name_to_ids)} manager names.")

    # ---------------- Slice Selected Manager ----------------
    def slice_selected_manager(self):
        name = self.manager_cb.get()
        out_folder = self.output_folder_var.get()
        if not name:
            messagebox.showerror("Error", "Please select a manager name")
            return
        if not out_folder:
            messagebox.showerror("Error", "Please select output folder")
            return

        ids = self.name_to_ids.get(name, [])
        if len(ids) > 1:
            # If multiple IDs for same name, ask user to select one
            id_select_win = tk.Toplevel(self.master)
            id_select_win.title(f"Select Manager ID for {name}")
            tk.Label(id_select_win, text=f"Select Manager ID for {name}:").pack(pady=5)
            selected_id_var = tk.StringVar()
            cb = ttk.Combobox(id_select_win, values=ids, textvariable=selected_id_var, state="readonly")
            cb.pack(pady=5)
            def confirm_id():
                manager_id = selected_id_var.get()
                if manager_id:
                    self.generate_slice(manager_id, name, out_folder)
                    id_select_win.destroy()
            tk.Button(id_select_win, text="Confirm", command=confirm_id).pack(pady=5)
            return
        elif len(ids) == 0:
            messagebox.showerror("Error", f"No manager ID found for {name}")
            return
        else:
            manager_id = ids[0]
            self.generate_slice(manager_id, name, out_folder)

    # ---------------- Slice All at GCB Level ----------------
    def slice_all_at_level(self):
        level = self.gcb_level_cb.get()
        out_folder = self.output_folder_var.get()
        if not level:
            messagebox.showerror("Error", "Please select a GCB level")
            return
        if not out_folder:
            messagebox.showerror("Error", "Please select output folder")
            return
        level = int(level)
        managers = self.df[self.df['Global Career Band']==level][['Employee ID','Employee Name']].drop_duplicates()
        for _, row in managers.iterrows():
            self.generate_slice(row['Employee ID'], row['Employee Name'], out_folder)

    # ---------------- Generate Slice ----------------
    def generate_slice(self, manager_id, manager_name, out_folder):
        sliced_df = get_hierarchy_slice(self.df, manager_id)
        if sliced_df.empty:
            messagebox.showinfo("Info", f"No records found under {manager_name}")
            return
        # Summary sheet
        total_emp = len(sliced_df)
        total_gcb3 = len(sliced_df[sliced_df['Global Career Band']==3])
        total_gcb4 = len(sliced_df[sliced_df['Global Career Band']==4])
        per_gcb3 = sliced_df[sliced_df['Global Career Band']==3].groupby('Employee Name')['Employee ID'].count().reset_index(name='Count')
        per_gcb4 = sliced_df[sliced_df['Global Career Band']==4].groupby('Employee Name')['Employee ID'].count().reset_index(name='Count')

        # Save Excel
        file_path = os.path.join(out_folder, f"{manager_name}_slice.xlsx")
        with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
            sliced_df.to_excel(writer, index=False, sheet_name="Hierarchy Slice")
            # Summary stacked
            start_row = 0
            summary = pd.DataFrame([["Manager Name", manager_name], ["Manager ID", manager_id], ["Total Employees", total_emp]])
            summary.to_excel(writer, index=False, header=False, sheet_name="Summary", startrow=start_row)
            start_row += len(summary) + 1
            pd.DataFrame([["Total GCB3", total_gcb3], ["Total GCB4", total_gcb4]]).to_excel(writer, index=False, header=False, sheet_name="Summary", startrow=start_row)
            start_row += 3
            per_gcb3.to_excel(writer, index=False, sheet_name="Summary", startrow=start_row)
            start_row += len(per_gcb3) + 2
            per_gcb4.to_excel(writer, index=False, sheet_name="Summary", startrow=start_row)
        self.status_var.set(f"Slice for {manager_name} saved → {file_path}")
        messagebox.showinfo("Success", f"Slice for {manager_name} saved successfully!")

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    root = tk.Tk()
    app = Phase3GUI(root)
    root.mainloop()


In [ ]:
# -------------------------
# Phase 2: Build Hierarchy (fixed missing-manager name resolution)
# -------------------------
def phase_two(input_file="phase1_enriched.xlsx", output_file="hierarchy_report.xlsx"):
    """Build flattened hierarchy with unlimited levels + summary + exceptions."""

    enriched = pd.read_excel(input_file, sheet_name="Enriched")
    enriched.columns = enriched.columns.str.strip()

    # Normalize ID/name columns as strings for robust matching
    enriched = enriched.copy()
    if EMP_ID_COL in enriched.columns:
        enriched[EMP_ID_COL] = enriched[EMP_ID_COL].astype(str).str.strip()
    if MGR_ID_COL in enriched.columns:
        enriched[MGR_ID_COL] = enriched[MGR_ID_COL].astype(str).str.strip()
    if EMP_NAME_COL in enriched.columns:
        enriched[EMP_NAME_COL] = enriched[EMP_NAME_COL].astype(str).str.strip()
    if MGR_NAME_COL in enriched.columns:
        enriched[MGR_NAME_COL] = enriched[MGR_NAME_COL].astype(str).str.strip()
    if GCB_COL in enriched.columns:
        enriched[GCB_COL] = enriched[GCB_COL].astype(str).str.strip().str.upper()

    # Build a DataFrame lookup and a deduplicated map for safe access
    df_lookup = enriched.set_index(EMP_ID_COL)

    hierarchy = []
    exceptions = []

    # helper to safely get a single row (Series) for an employee id (handles duplicates)
    def get_emp_row(emp_id):
        """Return a Series for emp_id if present, else None. Handles duplicate index by returning first row."""
        try:
            row = df_lookup.loc[emp_id]
        except KeyError:
            return None
        # if multiple rows returned (DataFrame), take first
        if isinstance(row, pd.DataFrame):
            return row.iloc[0]
        return row

    def recurse(manager_id, path):
        """Recursively walk down the hierarchy from a manager."""
        if pd.isna(manager_id) or manager_id == "":
            return
        reports = enriched[enriched[MGR_ID_COL] == manager_id]
        for _, row in reports.iterrows():
            record = path.copy()
            record.update({
                "Manager ID": manager_id,
                "Manager Name": row.get(MGR_NAME_COL, ""),
                "Manager GCB": get_emp_row(manager_id)[GCB_COL]
                if get_emp_row(manager_id) is not None else None,
            })
            # keep ALL columns from enriched
            record.update(row.to_dict())
            hierarchy.append(record)

            # recurse further down
            recurse(row[EMP_ID_COL], record)

    # Find all MDs
    mds = enriched[enriched[GCB_COL] == "MD"]
    for _, md in mds.iterrows():
        md_path = {"MD ID": md[EMP_ID_COL], "MD Name": md[EMP_NAME_COL]}

        # add MD itself
        row_dict = md.to_dict()
        row_dict.update({
            "MD ID": md[EMP_ID_COL],
            "MD Name": md[EMP_NAME_COL],
            "Manager ID": None,
            "Manager Name": None,
            "Manager GCB": None,
        })
        hierarchy.append(row_dict)

        # recurse into MD’s reports
        recurse(md[EMP_ID_COL], md_path)

    # ------------------------------
    # Exceptions: improved missing-manager tracing + proper missing manager name lookup
    # ------------------------------
    # build set of all known employee ids (strings)
    all_emp_ids = set(enriched[EMP_ID_COL].astype(str).tolist())

    # helper to resolve missing manager name for a missing manager id:
    def resolve_missing_manager_name(missing_mgr_id):
        """
        Try multiple ways to find a name for missing_mgr_id:
         1) If missing_mgr_id appears as an Employee ID (unexpected here) use employee name.
         2) Else, look for rows where MGR_ID_COL == missing_mgr_id and extract the MGR_NAME_COL values
            (these are subordinate rows that often carry the manager name).
         3) Else, return empty string.
        """
        missing_mgr_id = str(missing_mgr_id).strip()
        # 1) check if present as an employee (rare, since missing means not in all_emp_ids)
        emp_row = get_emp_row(missing_mgr_id)
        if emp_row is not None:
            return str(emp_row.get(EMP_NAME_COL, "")).strip()

        # 2) look for subordinates that reference this manager id and read the manager name field
        if MGR_NAME_COL in enriched.columns:
            candidate_names = enriched.loc[enriched[MGR_ID_COL] == missing_mgr_id, MGR_NAME_COL] \
                .dropna().astype(str).str.strip()
            if not candidate_names.empty:
                # return most common (mode) name if available, else first
                try:
                    mode_name = candidate_names.mode()
                    if not mode_name.empty:
                        return mode_name.iloc[0]
                except Exception:
                    pass
                return candidate_names.iloc[0]
        # 3) nothing found
        return ""

    # For each employee row, walk up until MD or until first missing manager ID encountered
    for _, row in enriched.iterrows():
        # skip MDs
        if str(row.get(GCB_COL, "")).strip().upper() == "MD":
            continue

        current_mgr_id = str(row.get(MGR_ID_COL, "")).strip()
        missing_mgr_id = None
        missing_mgr_name = ""
        chain_parts = []  # for optional trace like "A -> B -> C (MISSING)"

        # If no manager id present at all, treat as missing immediately
        if current_mgr_id == "" or pd.isna(current_mgr_id):
            missing_mgr_id = current_mgr_id
            missing_mgr_name = row.get(MGR_NAME_COL, "") or ""
        else:
            # climb upwards until MD or missing manager
            visited = set()
            while current_mgr_id:
                # protect from infinite loops
                if current_mgr_id in visited:
                    # circularity - mark as missing (use current_mgr_id)
                    missing_mgr_id = current_mgr_id
                    missing_mgr_name = resolve_missing_manager_name(current_mgr_id)
                    chain_parts.append(f"{missing_mgr_name or missing_mgr_id} (CIRCULAR)")
                    break
                visited.add(current_mgr_id)

                chain_parts.append(current_mgr_id)

                # if manager id not found in enriched -> this is the first missing manager
                if current_mgr_id not in all_emp_ids:
                    missing_mgr_id = current_mgr_id
                    missing_mgr_name = resolve_missing_manager_name(current_mgr_id)
                    chain_parts[-1] = f"{missing_mgr_name or missing_mgr_id} (MISSING)"
                    break

                # otherwise get manager row and move up
                mgr_row = get_emp_row(current_mgr_id)
                if mgr_row is None:
                    # should not happen (we checked membership), but safe-break
                    missing_mgr_id = current_mgr_id
                    missing_mgr_name = resolve_missing_manager_name(current_mgr_id)
                    break

                mgr_gcb = str(mgr_row.get(GCB_COL, "")).strip().upper()
                # if this manager is MD, chain completes successfully
                if mgr_gcb == "MD":
                    missing_mgr_id = None
                    missing_mgr_name = ""
                    break

                # else continue up
                next_mgr = mgr_row.get(MGR_ID_COL, "")
                current_mgr_id = str(next_mgr).strip() if pd.notna(next_mgr) else ""
            # end while

        if missing_mgr_id:
            rec = row.to_dict()
            rec["Missing Manager ID"] = missing_mgr_id
            rec["Missing Manager Name"] = missing_mgr_name
            rec["Missing Chain"] = " -> ".join(chain_parts)
            exceptions.append(rec)

    # Convert to DataFrames
    hierarchy_df = pd.DataFrame(hierarchy)
    exceptions_df = pd.DataFrame(exceptions)

    # -------- Single consolidated Summary sheet --------
    summary_tables = []

    # overall counts
    summary = pd.DataFrame([{
        "Metric": "Total Employees", "Value": len(enriched)
    }, {
        "Metric": "Total in Hierarchy", "Value": hierarchy_df[EMP_ID_COL].nunique()
    }, {
        "Metric": "Missing Managers Count", "Value": len(exceptions_df)
    }])
    summary_tables.append(("Overall Summary", summary))

    # per-MD
    per_md = hierarchy_df.groupby("MD Name")[EMP_ID_COL].nunique().reset_index()
    per_md.columns = ["MD Name", "Headcount"]
    summary_tables.append(("Headcount per MD", per_md))

    # ---- Prepare a safe manager-name map from enriched (string keys) ----
    manager_name_map = {}
    if EMP_ID_COL in enriched.columns and EMP_NAME_COL in enriched.columns:
        mgr_df = enriched[[EMP_ID_COL, EMP_NAME_COL]].drop_duplicates().copy()
        mgr_df[EMP_ID_COL] = mgr_df[EMP_ID_COL].astype(str).str.strip()
        mgr_df[EMP_NAME_COL] = mgr_df[EMP_NAME_COL].astype(str).str.strip()
        manager_name_map = mgr_df.set_index(EMP_ID_COL)[EMP_NAME_COL].to_dict()

    # Ensure Manager GCB column exists
    if "Manager GCB" not in hierarchy_df.columns:
        hierarchy_df["Manager GCB"] = pd.NA

    mgr_gcb_numeric = pd.to_numeric(hierarchy_df["Manager GCB"], errors="coerce")

    # -------------------------------
    # Per-GCB3 direct report counts
    # -------------------------------
    per_gcb3_counts = (
        hierarchy_df.loc[mgr_gcb_numeric == 3]
        .groupby("Manager ID")[EMP_ID_COL]
        .nunique()
        .reset_index(name="Direct Reports")
    )
    per_gcb3_counts["Manager ID"] = per_gcb3_counts["Manager ID"].astype(str).str.strip()
    per_gcb3_counts["GCB3 Name"] = per_gcb3_counts["Manager ID"].map(manager_name_map).fillna("")
    per_gcb3 = per_gcb3_counts[["GCB3 Name", "Manager ID", "Direct Reports"]].copy()
    per_gcb3.columns = ["GCB3 Name", "GCB3 ID", "Direct Reports"]
    summary_tables.append(("Direct Reports per GCB3", per_gcb3))

    # -------------------------------
    # Per-GCB4 direct report counts
    # -------------------------------
    per_gcb4_counts = (
        hierarchy_df.loc[mgr_gcb_numeric == 4]
        .groupby("Manager ID")[EMP_ID_COL]
        .nunique()
        .reset_index(name="Direct Reports")
    )
    per_gcb4_counts["Manager ID"] = per_gcb4_counts["Manager ID"].astype(str).str.strip()
    per_gcb4_counts["GCB4 Name"] = per_gcb4_counts["Manager ID"].map(manager_name_map).fillna("")
    per_gcb4 = per_gcb4_counts[["GCB4 Name", "Manager ID", "Direct Reports"]].copy()
    per_gcb4.columns = ["GCB4 Name", "GCB4 ID", "Direct Reports"]
    summary_tables.append(("Direct Reports per GCB4", per_gcb4))

    # -------------------------------
    # WRITE OUTPUT (Hierarchy + Missing Managers + ONE Summary sheet stacked)
    # -------------------------------
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        hierarchy_df.to_excel(writer, index=False, sheet_name="Hierarchy Report")
        if not exceptions_df.empty:
            exceptions_df.to_excel(writer, index=False, sheet_name="Missing Managers")

        # Write all summaries stacked in one sheet with title rows
        start_row = 0
        for title, df in summary_tables:
            pd.DataFrame([[title]]).to_excel(writer, index=False, header=False,
                                             sheet_name="Summary", startrow=start_row)
            start_row += 1
            df.to_excel(writer, index=False, sheet_name="Summary", startrow=start_row)
            start_row += len(df) + 2

    print(f"✅ Phase 2 completed → {output_file}")
    return hierarchy_df, exceptions_df


In [ ]:
# -------------------------
# Phase 2: Build Hierarchy
# -------------------------
def phase_two(input_file="phase1_enriched.xlsx", output_file="hierarchy_report.xlsx"):
    """Build flattened hierarchy with unlimited levels + summary + exceptions."""

    enriched = pd.read_excel(input_file, sheet_name="Enriched")
    enriched.columns = enriched.columns.str.strip()

    df_lookup = enriched.set_index(EMP_ID_COL)

    hierarchy = []
    exceptions = []

    def recurse(manager_id, path):
        """Recursively walk down the hierarchy from a manager."""
        reports = enriched[enriched[MGR_ID_COL] == manager_id]
        for _, row in reports.iterrows():
            record = path.copy()
            record.update({
                "Manager ID": manager_id,
                "Manager Name": row[MGR_NAME_COL],
                "Manager GCB": df_lookup.loc[manager_id, GCB_COL]
                if manager_id in df_lookup.index else None,
            })
            # keep ALL columns from enriched
            record.update(row.to_dict())
            hierarchy.append(record)

            # recurse further down
            recurse(row[EMP_ID_COL], record)

    # Find all MDs
    mds = enriched[enriched[GCB_COL] == "MD"]
    for _, md in mds.iterrows():
        md_path = {"MD ID": md[EMP_ID_COL], "MD Name": md[EMP_NAME_COL]}

        # add MD itself
        row_dict = md.to_dict()
        row_dict.update({
            "MD ID": md[EMP_ID_COL],
            "MD Name": md[EMP_NAME_COL],
            "Manager ID": None,
            "Manager Name": None,
            "Manager GCB": None,
        })
        hierarchy.append(row_dict)

        # recurse into MD’s reports
        recurse(md[EMP_ID_COL], md_path)

    # -------------------------------
    # Exceptions: trace up the chain until first missing manager is found
    # -------------------------------
    all_emp_ids = set(enriched[EMP_ID_COL])

    for _, row in enriched.iterrows():
        if row[GCB_COL] == "MD":
            continue  # MDs are top, never exceptions

        current_mgr_id = row[MGR_ID_COL]
        missing_mgr_id = None
        missing_mgr_name = None

        # climb upwards until we either reach MD or hit a missing manager
        while pd.notna(current_mgr_id):
            if current_mgr_id not in all_emp_ids:
                missing_mgr_id = current_mgr_id
                # take the manager name field from this row (the direct manager not found)
                missing_mgr_name = row.get(MGR_NAME_COL, None)
                break

            # move to next manager in chain
            mgr_row = enriched.loc[enriched[EMP_ID_COL] == current_mgr_id]
            if mgr_row.empty:
                break
            current_mgr_id = mgr_row.iloc[0][MGR_ID_COL]

        if missing_mgr_id:
            rec = row.to_dict()
            rec["Missing Manager ID"] = missing_mgr_id
            rec["Missing Manager Name"] = missing_mgr_name
            exceptions.append(rec)

    hierarchy_df = pd.DataFrame(hierarchy)
    exceptions_df = pd.DataFrame(exceptions)

    # -------- Single consolidated Summary sheet --------
    summary_tables = []

    # overall counts
    summary = pd.DataFrame([{
        "Metric": "Total Employees", "Value": len(enriched)
    }, {
        "Metric": "Total in Hierarchy", "Value": hierarchy_df[EMP_ID_COL].nunique()
    }, {
        "Metric": "Missing Managers Count", "Value": len(exceptions_df)
    }])
    summary_tables.append(("Overall Summary", summary))

    # per-MD
    per_md = hierarchy_df.groupby("MD Name")[EMP_ID_COL].nunique().reset_index()
    per_md.columns = ["MD Name", "Headcount"]
    summary_tables.append(("Headcount per MD", per_md))

    # ---- Prepare a safe manager-name map from enriched (string keys) ----
    manager_name_map = {}
    if EMP_ID_COL in enriched.columns and EMP_NAME_COL in enriched.columns:
        mgr_df = enriched[[EMP_ID_COL, EMP_NAME_COL]].drop_duplicates().copy()
        mgr_df[EMP_ID_COL] = mgr_df[EMP_ID_COL].astype(str).str.strip()
        mgr_df[EMP_NAME_COL] = mgr_df[EMP_NAME_COL].astype(str).str.strip()
        manager_name_map = mgr_df.set_index(EMP_ID_COL)[EMP_NAME_COL].to_dict()

    # Make sure Manager GCB column exists in hierarchy_df (if not, create safe NA column)
    if "Manager GCB" not in hierarchy_df.columns:
        hierarchy_df["Manager GCB"] = pd.NA

    # Convert Manager GCB to numeric where possible for correct matching (coerce errors)
    mgr_gcb_numeric = pd.to_numeric(hierarchy_df["Manager GCB"], errors="coerce")

    # -------------------------------
    # Per-GCB3: direct report counts
    # -------------------------------
    per_gcb3_counts = (
        hierarchy_df.loc[mgr_gcb_numeric == 3]
        .groupby("Manager ID")[EMP_ID_COL]
        .nunique()
        .reset_index(name="Direct Reports")
    )
    per_gcb3_counts["Manager ID"] = per_gcb3_counts["Manager ID"].astype(str).str.strip()
    per_gcb3_counts["GCB3 Name"] = per_gcb3_counts["Manager ID"].map(manager_name_map).fillna("")
    per_gcb3 = per_gcb3_counts[["GCB3 Name", "Manager ID", "Direct Reports"]].copy()
    per_gcb3.columns = ["GCB3 Name", "GCB3 ID", "Direct Reports"]
    summary_tables.append(("Direct Reports per GCB3", per_gcb3))

    # -------------------------------
    # Per-GCB4: direct report counts
    # -------------------------------
    per_gcb4_counts = (
        hierarchy_df.loc[mgr_gcb_numeric == 4]
        .groupby("Manager ID")[EMP_ID_COL]
        .nunique()
        .reset_index(name="Direct Reports")
    )
    per_gcb4_counts["Manager ID"] = per_gcb4_counts["Manager ID"].astype(str).str.strip()
    per_gcb4_counts["GCB4 Name"] = per_gcb4_counts["Manager ID"].map(manager_name_map).fillna("")
    per_gcb4 = per_gcb4_counts[["GCB4 Name", "Manager ID", "Direct Reports"]].copy()
    per_gcb4.columns = ["GCB4 Name", "GCB4 ID", "Direct Reports"]
    summary_tables.append(("Direct Reports per GCB4", per_gcb4))

    # -------------------------------
    # WRITE OUTPUT (Hierarchy + Missing Managers + ONE Summary sheet stacked)
    # -------------------------------
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        hierarchy_df.to_excel(writer, index=False, sheet_name="Hierarchy Report")
        if not exceptions_df.empty:
            exceptions_df.to_excel(writer, index=False, sheet_name="Missing Managers")

        # Write all summaries stacked in one sheet with title rows
        start_row = 0
        for title, df in summary_tables:
            pd.DataFrame([[title]]).to_excel(writer, index=False, header=False,
                                             sheet_name="Summary", startrow=start_row)
            start_row += 1
            df.to_excel(writer, index=False, sheet_name="Summary", startrow=start_row)
            start_row += len(df) + 2

    print(f"✅ Phase 2 completed → {output_file}")
    return hierarchy_df, exceptions_df


In [ ]:
# -------------------------
# Phase 2: Build Hierarchy (updated missing-manager tracing)
# -------------------------
def phase_two(input_file="phase1_enriched.xlsx", output_file="hierarchy_report.xlsx"):
    """Build flattened hierarchy with unlimited levels + summary + exceptions (improved missing-manager tracing)."""

    enriched = pd.read_excel(input_file, sheet_name="Enriched")
    enriched.columns = enriched.columns.str.strip()

    # NORMALIZE key columns to strings for reliable matching
    enriched = enriched.copy()
    enriched[EMP_ID_COL] = enriched[EMP_ID_COL].astype(str).str.strip()
    # If MGR_ID_COL not present, create empty column
    if MGR_ID_COL not in enriched.columns:
        enriched[MGR_ID_COL] = ""
    else:
        enriched[MGR_ID_COL] = enriched[MGR_ID_COL].astype(str).str.strip()
    # Normalize names and GCB for safety
    enriched[EMP_NAME_COL] = enriched[EMP_NAME_COL].astype(str).str.strip()
    if MGR_NAME_COL in enriched.columns:
        enriched[MGR_NAME_COL] = enriched[MGR_NAME_COL].astype(str).str.strip()
    if GCB_COL in enriched.columns:
        enriched[GCB_COL] = enriched[GCB_COL].astype(str).str.strip().str.upper()

    # Build lookup using normalized Employee ID
    df_lookup = enriched.set_index(EMP_ID_COL)

    hierarchy = []
    exceptions = []

    def recurse(manager_id, path):
        """Recursively walk down the hierarchy from a manager (manager_id must be normalized string)."""
        if manager_id is None:
            return
        manager_id = str(manager_id).strip()
        reports = enriched[enriched[MGR_ID_COL] == manager_id]
        for _, row in reports.iterrows():
            record = path.copy()
            record.update({
                "Manager ID": manager_id,
                "Manager Name": row.get(MGR_NAME_COL, ""),
                "Manager GCB": df_lookup.loc[manager_id, GCB_COL]
                if manager_id in df_lookup.index else None,
            })
            # keep ALL columns from enriched
            record.update(row.to_dict())
            hierarchy.append(record)

            # recurse further down
            recurse(row[EMP_ID_COL], record)

    # Find all MDs (normalized)
    mds = enriched[enriched[GCB_COL] == "MD"]
    for _, md in mds.iterrows():
        md_path = {"MD ID": md[EMP_ID_COL], "MD Name": md[EMP_NAME_COL]}

        # add MD itself (full columns)
        row_dict = md.to_dict()
        row_dict.update({
            "MD ID": md[EMP_ID_COL],
            "MD Name": md[EMP_NAME_COL],
            "Manager ID": "",
            "Manager Name": "",
            "Manager GCB": "",
        })
        hierarchy.append(row_dict)

        # recurse into MD’s reports
        recurse(md[EMP_ID_COL], md_path)

    # ------------------------------
    # Exceptions: improved missing-manager tracing
    # ------------------------------
    # helper: find first missing manager in the upward chain for a given employee row
    def find_first_missing_manager(emp_row):
        """
        Walk upwards from emp_row[MGR_ID_COL] and:
         - return (missing_mgr_id, missing_mgr_name, chain_str) when first missing manager encountered
         - return (None, None, chain_str) if chain reaches MD (i.e. no missing found)
        """
        visited = set()
        chain = []  # collect tuples (id, name_or_empty)
        current_mgr = str(emp_row.get(MGR_ID_COL, "")).strip()

        while current_mgr:
            # avoid infinite loops
            if current_mgr in visited:
                chain.append((current_mgr, "(circular)"))
                return current_mgr, "", " -> ".join(f"{i}({n})" for i, n in chain)
            visited.add(current_mgr)

            # manager exists in enriched? continue upward
            if current_mgr in df_lookup.index:
                mgr_row = df_lookup.loc[current_mgr]
                mgr_name = str(mgr_row.get(EMP_NAME_COL, "")).strip()
                chain.append((current_mgr, mgr_name or ""))
                # if this manager is MD, chain completes successfully (no missing)
                mgr_gcb = str(mgr_row.get(GCB_COL, "")).strip().upper()
                if mgr_gcb == "MD":
                    # build chain string and indicate no missing
                    chain_str = " -> ".join(f"{n or i}" for i, n in chain)
                    return None, None, chain_str
                # move up
                next_mgr = mgr_row.get(MGR_ID_COL, "")
                current_mgr = str(next_mgr).strip() if pd.notna(next_mgr) else ""
            else:
                # current_mgr is not found in enriched -> this is the first missing manager
                # try to recover a name for this missing manager from any rows that reference them
                # (many employees might have MGR_ID == current_mgr and have the manager's name in MGR_NAME_COL)
                possible_names = enriched.loc[enriched[MGR_ID_COL] == current_mgr, MGR_NAME_COL] \
                                 if MGR_NAME_COL in enriched.columns else pd.Series([], dtype=object)
                possible_names = possible_names.dropna().astype(str).str.strip()
                missing_name = possible_names.mode().iloc[0] if not possible_names.empty else ""
                chain.append((current_mgr, missing_name))
                chain_str = " -> ".join(f"{n or i}" for i, n in chain) + " (MISSING)"
                return current_mgr, missing_name, chain_str

        # if we exit loop without encountering MD or missing (e.g., no manager)
        return None, None, "No manager chain"

    # Apply find_first_missing_manager to every employee row and capture exceptions
    for _, row in enriched.iterrows():
        missing_id, missing_name, chain_str = find_first_missing_manager(row)
        if missing_id:  # found a missing manager somewhere above this employee
            row_dict = row.to_dict()
            row_dict["Missing Manager ID"] = missing_id
            row_dict["Missing Manager Name"] = missing_name
            row_dict["Missing Chain"] = chain_str
            exceptions.append(row_dict)

    hierarchy_df = pd.DataFrame(hierarchy)
    exceptions_df = pd.DataFrame(exceptions)

    # -------- Single consolidated Summary sheet (unchanged logic) --------
    summary_tables = []

    # overall counts
    summary = pd.DataFrame([{
        "Metric": "Total Employees", "Value": len(enriched)
    }, {
        "Metric": "Total in Hierarchy", "Value": hierarchy_df[EMP_ID_COL].nunique()
    }, {
        "Metric": "Missing Managers Count", "Value": len(exceptions_df)
    }])
    summary_tables.append(("Overall Summary", summary))

    # per-MD
    per_md = hierarchy_df.groupby("MD Name")[EMP_ID_COL].nunique().reset_index()
    per_md.columns = ["MD Name", "Headcount"]
    summary_tables.append(("Headcount per MD", per_md))

    # ---- Prepare manager-name map ----
    manager_name_map = {}
    if EMP_ID_COL in enriched.columns and EMP_NAME_COL in enriched.columns:
        mgr_df = enriched[[EMP_ID_COL, EMP_NAME_COL]].drop_duplicates().copy()
        mgr_df[EMP_ID_COL] = mgr_df[EMP_ID_COL].astype(str).str.strip()
        mgr_df[EMP_NAME_COL] = mgr_df[EMP_NAME_COL].astype(str).str.strip()
        manager_name_map = mgr_df.set_index(EMP_ID_COL)[EMP_NAME_COL].to_dict()

    # Ensure Manager GCB column exists
    if "Manager GCB" not in hierarchy_df.columns:
        hierarchy_df["Manager GCB"] = pd.NA

    mgr_gcb_numeric = pd.to_numeric(hierarchy_df["Manager GCB"], errors="coerce")

    # -------------------------------
    # Per-GCB3 direct report counts
    # -------------------------------
    per_gcb3_counts = (
        hierarchy_df.loc[mgr_gcb_numeric == 3]
        .groupby("Manager ID")[EMP_ID_COL]
        .nunique()
        .reset_index(name="Direct Reports")
    )
    per_gcb3_counts["Manager ID"] = per_gcb3_counts["Manager ID"].astype(str).str.strip()
    per_gcb3_counts["GCB3 Name"] = per_gcb3_counts["Manager ID"].map(manager_name_map).fillna("")
    per_gcb3 = per_gcb3_counts[["GCB3 Name", "Manager ID", "Direct Reports"]].copy()
    per_gcb3.columns = ["GCB3 Name", "GCB3 ID", "Direct Reports"]
    summary_tables.append(("Direct Reports per GCB3", per_gcb3))

    # -------------------------------
    # Per-GCB4 direct report counts
    # -------------------------------
    per_gcb4_counts = (
        hierarchy_df.loc[mgr_gcb_numeric == 4]
        .groupby("Manager ID")[EMP_ID_COL]
        .nunique()
        .reset_index(name="Direct Reports")
    )
    per_gcb4_counts["Manager ID"] = per_gcb4_counts["Manager ID"].astype(str).str.strip()
    per_gcb4_counts["GCB4 Name"] = per_gcb4_counts["Manager ID"].map(manager_name_map).fillna("")
    per_gcb4 = per_gcb4_counts[["GCB4 Name", "Manager ID", "Direct Reports"]].copy()
    per_gcb4.columns = ["GCB4 Name", "GCB4 ID", "Direct Reports"]
    summary_tables.append(("Direct Reports per GCB4", per_gcb4))

    # -------------------------------
    # WRITE OUTPUT
    # -------------------------------
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        hierarchy_df.to_excel(writer, index=False, sheet_name="Hierarchy Report")
        if not exceptions_df.empty:
            exceptions_df.to_excel(writer, index=False, sheet_name="Missing Managers")

        # Write summaries stacked in one sheet
        start_row = 0
        for title, df in summary_tables:
            pd.DataFrame([[title]]).to_excel(writer, index=False, header=False,
                                             sheet_name="Summary", startrow=start_row)
            start_row += 1
            df.to_excel(writer, index=False, sheet_name="Summary", startrow=start_row)
            start_row += len(df) + 2

    print(f"✅ Phase 2 completed → {output_file}")
    return hierarchy_df, exceptions_df


In [ ]:
# -------------------------
# Phase 2: Build Hierarchy
# -------------------------
def phase_two(input_file="phase1_enriched.xlsx", output_file="hierarchy_report.xlsx"):
    """Build flattened hierarchy with unlimited levels + summary + exceptions."""

    enriched = pd.read_excel(input_file, sheet_name="Enriched")
    enriched.columns = enriched.columns.str.strip()

    df_lookup = enriched.set_index(EMP_ID_COL)

    hierarchy = []
    exceptions = []

    def recurse(manager_id, path):
        """Recursively walk down the hierarchy from a manager."""
        reports = enriched[enriched[MGR_ID_COL] == manager_id]
        for _, row in reports.iterrows():
            record = path.copy()
            record.update({
                "Manager ID": manager_id,
                "Manager Name": row[MGR_NAME_COL],
                "Manager GCB": df_lookup.loc[manager_id, GCB_COL]
                if manager_id in df_lookup.index else None,
            })
            # keep ALL columns from enriched
            record.update(row.to_dict())
            hierarchy.append(record)

            # recurse further down
            recurse(row[EMP_ID_COL], record)

    # Find all MDs
    mds = enriched[enriched[GCB_COL] == "MD"]
    for _, md in mds.iterrows():
        md_path = {"MD ID": md[EMP_ID_COL], "MD Name": md[EMP_NAME_COL]}

        # add MD itself
        row_dict = md.to_dict()
        row_dict.update({
            "MD ID": md[EMP_ID_COL],
            "MD Name": md[EMP_NAME_COL],
            "Manager ID": None,
            "Manager Name": None,
            "Manager GCB": None,
        })
        hierarchy.append(row_dict)

        # recurse into MD’s reports
        recurse(md[EMP_ID_COL], md_path)

    # Exceptions: employees whose manager ID not in employee list
    all_emp_ids = set(enriched[EMP_ID_COL])
    missing_mgr_ids = set(enriched[MGR_ID_COL]) - all_emp_ids
    for _, row in enriched[enriched[MGR_ID_COL].isin(missing_mgr_ids)].iterrows():
        if row[GCB_COL] != "MD":
            exceptions.append(row.to_dict())

    hierarchy_df = pd.DataFrame(hierarchy)
    exceptions_df = pd.DataFrame(exceptions)

    # -------- Single consolidated Summary sheet --------
      # -------- Single consolidated Summary sheet (REPLACE EXISTING SUMMARY BLOCK WITH THIS) --------
    summary_tables = []

    # overall counts
    summary = pd.DataFrame([{
        "Metric": "Total Employees", "Value": len(enriched)
    }, {
        "Metric": "Total in Hierarchy", "Value": hierarchy_df[EMP_ID_COL].nunique()
    }, {
        "Metric": "Missing Managers Count", "Value": len(exceptions_df)
    }])
    summary_tables.append(("Overall Summary", summary))

    # per-MD
    per_md = hierarchy_df.groupby("MD Name")[EMP_ID_COL].nunique().reset_index()
    per_md.columns = ["MD Name", "Headcount"]
    summary_tables.append(("Headcount per MD", per_md))

    # ---- Prepare a safe manager-name map from enriched (string keys) ----
    manager_name_map = {}
    if EMP_ID_COL in enriched.columns and EMP_NAME_COL in enriched.columns:
        mgr_df = enriched[[EMP_ID_COL, EMP_NAME_COL]].drop_duplicates().copy()
        mgr_df[EMP_ID_COL] = mgr_df[EMP_ID_COL].astype(str).str.strip()
        mgr_df[EMP_NAME_COL] = mgr_df[EMP_NAME_COL].astype(str).str.strip()
        manager_name_map = mgr_df.set_index(EMP_ID_COL)[EMP_NAME_COL].to_dict()

    # Make sure Manager GCB column exists in hierarchy_df (if not, create safe NA column)
    if "Manager GCB" not in hierarchy_df.columns:
        hierarchy_df["Manager GCB"] = pd.NA

    # Convert Manager GCB to numeric where possible for correct matching (coerce errors)
    mgr_gcb_numeric = pd.to_numeric(hierarchy_df["Manager GCB"], errors="coerce")

    # -------------------------------
    # Per-GCB3: direct report counts
    # -------------------------------
    per_gcb3_counts = (
        hierarchy_df.loc[mgr_gcb_numeric == 3]
        .groupby("Manager ID")[EMP_ID_COL]
        .nunique()
        .reset_index(name="Direct Reports")
    )

    # normalize Manager ID as string to map reliably
    per_gcb3_counts["Manager ID"] = per_gcb3_counts["Manager ID"].astype(str).str.strip()
    per_gcb3_counts["GCB3 Name"] = per_gcb3_counts["Manager ID"].map(manager_name_map).fillna("")
    per_gcb3 = per_gcb3_counts[["GCB3 Name", "Manager ID", "Direct Reports"]].copy()
    per_gcb3.columns = ["GCB3 Name", "GCB3 ID", "Direct Reports"]
    summary_tables.append(("Direct Reports per GCB3", per_gcb3))

    # -------------------------------
    # Per-GCB4: direct report counts
    # -------------------------------
    per_gcb4_counts = (
        hierarchy_df.loc[mgr_gcb_numeric == 4]
        .groupby("Manager ID")[EMP_ID_COL]
        .nunique()
        .reset_index(name="Direct Reports")
    )
    per_gcb4_counts["Manager ID"] = per_gcb4_counts["Manager ID"].astype(str).str.strip()
    per_gcb4_counts["GCB4 Name"] = per_gcb4_counts["Manager ID"].map(manager_name_map).fillna("")
    per_gcb4 = per_gcb4_counts[["GCB4 Name", "Manager ID", "Direct Reports"]].copy()
    per_gcb4.columns = ["GCB4 Name", "GCB4 ID", "Direct Reports"]
    summary_tables.append(("Direct Reports per GCB4", per_gcb4))

    # -------------------------------
    # WRITE OUTPUT (Hierarchy + Missing Managers + ONE Summary sheet stacked)
    # -------------------------------
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        hierarchy_df.to_excel(writer, index=False, sheet_name="Hierarchy Report")
        if not exceptions_df.empty:
            exceptions_df.to_excel(writer, index=False, sheet_name="Missing Managers")

        # Write all summaries stacked in one sheet with title rows
        start_row = 0
        for title, df in summary_tables:
            # header/title row
            pd.DataFrame([[title]]).to_excel(writer, index=False, header=False,
                                             sheet_name="Summary", startrow=start_row)
            start_row += 1
            df.to_excel(writer, index=False, sheet_name="Summary", startrow=start_row)
            start_row += len(df) + 2  # leave a gap

    print(f"✅ Phase 2 completed → {output_file}")
    return hierarchy_df, exceptions_df


In [2]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE

# Create presentation
prs = Presentation()
slide_layout = prs.slide_layouts[6]  # blank slide
slide = prs.slides.add_slide(slide_layout)

# Title
left, top, width, height = Inches(0.5), Inches(0.2), Inches(9), Inches(1)
textbox = slide.shapes.add_textbox(left, top, width, height)
tf = textbox.text_frame
tf.text = "Cost Template Automation – Saving 144 Hours Annually"
p = tf.paragraphs[0]
p.font.size = Pt(28)
p.font.bold = True
p.font.color.rgb = RGBColor(0, 51, 102)

# Process flow boxes
process_steps = [
    ("Step 1: Data Preparation\nTool: CostTemplate_Data_Prep_Tool_KS", 0.5),
    ("Step 2: Report Generation\nTool: Cost_Templates_Gen_Tool_KS", 3.5),
    ("Step 3: Finalization\nTool: PasteAsValuesUtility", 6.5)
]

for text, left_in in process_steps:
    shape = slide.shapes.add_shape(
        MSO_SHAPE.ROUNDED_RECTANGLE,
        Inches(left_in), Inches(2), Inches(2.8), Inches(1.5)
    )
    shape.fill.solid()
    shape.fill.fore_color.rgb = RGBColor(91, 155, 213)
    shape.text = text
    for p in shape.text_frame.paragraphs:
        p.font.size = Pt(14)
        p.font.color.rgb = RGBColor(255, 255, 255)
        p.alignment = 1  # center

# Add arrows between steps
for i in range(2):
    slide.shapes.add_connector(
        1, Inches(3.3 + i*3), Inches(2.75), Inches(3.5 + i*3), Inches(2.75)
    )

# Benefits box
shape = slide.shapes.add_shape(
    MSO_SHAPE.RECTANGLE,
    Inches(0.5), Inches(4), Inches(8.5), Inches(2)
)
shape.fill.solid()
shape.fill.fore_color.rgb = RGBColor(237, 125, 49)
shape.text = (
    "Benefits:\n"
    "• Saves 12 hours per month (~144 hours annually)\n"
    "• Eliminates manual TM1 refreshes & errors\n"
    "• Ensures consistent, validated dashboards\n"
    "• Ready-to-share lightweight reports"
)
for p in shape.text_frame.paragraphs:
    p.font.size = Pt(16)
    p.font.color.rgb = RGBColor(255, 255, 255)

# Save file
prs.save("Cost_Automation_OnePager.pptx")
print("File Saved")

File Saved


In [ ]:
import pandas as pd

# -------------------------
# Constants
# -------------------------
EMP_ID_COL = "Employee ID"
EMP_NAME_COL = "Employee Name"
EMP_EMAIL_COL = "Employee Business Email Address"
MGR_ID_COL = "Entity Manager Employee ID"   # must exist in monthly.csv
MGR_NAME_COL = "Entity Manager Employee Name"
GCB_COL = "Global Career Band"


# -------------------------
# Phase 1: Enrichment
# -------------------------
def phase_one(gha_file, monthly_file, output_file="phase1_enriched.xlsx"):
    """Enrich monthly CSV with GHA details."""

    # Read monthly file (CSV may have special encoding)
    monthly = pd.read_csv(monthly_file, encoding="ISO-8859-1")
    gha = pd.read_excel(gha_file, sheet_name="Headcount - Employee Detail")

    # Strip column names
    monthly.columns = monthly.columns.str.strip()
    gha.columns = gha.columns.str.strip()

    # Select needed columns from GHA (add more here if needed)
    gha_subset = gha[
        [
            EMP_ID_COL,
            EMP_NAME_COL,
            EMP_EMAIL_COL,
            GCB_COL,
            "Company",
            "Department",
            "Job Function",
            "Legal Entity Name",
            "Employee Status"
        ]
    ].drop_duplicates()

    # Merge monthly + gha
    merged = monthly.merge(
        gha_subset,
        on=EMP_ID_COL,
        how="left",
        suffixes=("", "_GHA")
    )

    # Add Manager GCB by merging again on Manager ID
    mgr_gcb = gha_subset[[EMP_ID_COL, GCB_COL]].rename(
        columns={EMP_ID_COL: MGR_ID_COL, GCB_COL: "Manager GCB"}
    )
    merged = merged.merge(mgr_gcb, on=MGR_ID_COL, how="left")

    # Capture missing GHA matches
    missing = merged[merged[GCB_COL].isna()]

    # Save Phase 1
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        merged.to_excel(writer, index=False, sheet_name="Enriched")
        if not missing.empty:
            missing.to_excel(writer, index=False, sheet_name="Missing in GHA")

    print(f"✅ Phase 1 completed → {output_file}")
    return merged


# -------------------------
# Phase 2: Build Hierarchy
# -------------------------
def phase_two(input_file="phase1_enriched.xlsx", output_file="hierarchy_report.xlsx"):
    """Build flattened hierarchy with unlimited levels + summary + exceptions."""

    enriched = pd.read_excel(input_file, sheet_name="Enriched")
    enriched.columns = enriched.columns.str.strip()

    df_lookup = enriched.set_index(EMP_ID_COL)

    hierarchy = []
    exceptions = []

    def recurse(manager_id, path):
        """Recursively walk down the hierarchy from a manager."""
        reports = enriched[enriched[MGR_ID_COL] == manager_id]
        for _, row in reports.iterrows():
            record = path.copy()
            record.update({
                "Manager ID": manager_id,
                "Manager Name": row[MGR_NAME_COL],
                "Manager GCB": df_lookup.loc[manager_id, GCB_COL]
                if manager_id in df_lookup.index else None,
            })
            # keep ALL columns from enriched
            record.update(row.to_dict())
            hierarchy.append(record)

            # recurse further down
            recurse(row[EMP_ID_COL], record)

    # Find all MDs
    mds = enriched[enriched[GCB_COL] == "MD"]
    for _, md in mds.iterrows():
        md_path = {"MD ID": md[EMP_ID_COL], "MD Name": md[EMP_NAME_COL]}

        # add MD itself
        row_dict = md.to_dict()
        row_dict.update({
            "MD ID": md[EMP_ID_COL],
            "MD Name": md[EMP_NAME_COL],
            "Manager ID": None,
            "Manager Name": None,
            "Manager GCB": None,
        })
        hierarchy.append(row_dict)

        # recurse into MD’s reports
        recurse(md[EMP_ID_COL], md_path)

    # Exceptions: employees whose manager ID not in employee list
    all_emp_ids = set(enriched[EMP_ID_COL])
    missing_mgr_ids = set(enriched[MGR_ID_COL]) - all_emp_ids
    for _, row in enriched[enriched[MGR_ID_COL].isin(missing_mgr_ids)].iterrows():
        if row[GCB_COL] != "MD":
            exceptions.append(row.to_dict())

    hierarchy_df = pd.DataFrame(hierarchy)
    exceptions_df = pd.DataFrame(exceptions)

    # -------- Summary sheet --------
    summary = {}
    summary["Total Employees"] = len(enriched)
    summary["Total in Hierarchy"] = hierarchy_df[EMP_ID_COL].nunique()
    summary["Missing Managers Count"] = len(exceptions_df)

    per_md = hierarchy_df.groupby("MD Name")[EMP_ID_COL].nunique().reset_index()
    per_md.columns = ["MD Name", "Headcount"]

    per_gcb3 = hierarchy_df[hierarchy_df[GCB_COL] == 3] \
        .groupby(EMP_NAME_COL)[EMP_ID_COL].count().reset_index()
    per_gcb3.columns = ["GCB3 Name", "Direct Reports"]

    per_gcb4 = hierarchy_df[hierarchy_df[GCB_COL] == 4] \
        .groupby(EMP_NAME_COL)[EMP_ID_COL].count().reset_index()
    per_gcb4.columns = ["GCB4 Name", "Direct Reports"]

    # Save all outputs
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        hierarchy_df.to_excel(writer, index=False, sheet_name="Hierarchy Report")
        if not exceptions_df.empty:
            exceptions_df.to_excel(writer, index=False, sheet_name="Missing Managers")

        pd.DataFrame([summary]).to_excel(writer, index=False, sheet_name="Summary")
        per_md.to_excel(writer, index=False, sheet_name="MD Headcount")
        per_gcb3.to_excel(writer, index=False, sheet_name="GCB3 Reports")
        per_gcb4.to_excel(writer, index=False, sheet_name="GCB4 Reports")

    print(f"✅ Phase 2 completed → {output_file}")
    return hierarchy_df, exceptions_df


# -------------------------
# Run pipeline
# -------------------------
if __name__ == "__main__":
    gha_file = "GHA.xlsx"          # input GHA file
    csv_file = "Monthly.csv"       # input monthly file
    output_file_phase1 = "phase1_enriched.xlsx"
    final_output = "hierarchy_report.xlsx"

    print("Running Phase 1...")
    phase_one(gha_file, csv_file, output_file_phase1)

    print("Running Phase 2...")
    phase_two(output_file_phase1, final_output)

    print("✅ Processing complete!")


In [ ]:
import pandas as pd

# -------------------------
# Constants
# -------------------------
EMP_ID_COL = "Employee ID"
EMP_NAME_COL = "Employee Name"
EMP_EMAIL_COL = "Employee Business Email Address"
MGR_ID_COL = "Entity Manager Employee ID"   # must exist in monthly.csv
MGR_NAME_COL = "Entity Manager Employee Name"
GCB_COL = "Global Career Band"

# -------------------------
# Phase 1: Enrichment
# -------------------------
def enrich_monthly_with_gha(monthly_file, gha_file, output_file="phase1_enriched.xlsx"):
    """Enrich monthly CSV with GHA details."""

    # Read monthly file (CSV may have special encoding)
    monthly = pd.read_csv(monthly_file, encoding="ISO-8859-1")
    gha = pd.read_excel(gha_file, sheet_name="Headcount - Employee Detail")

    # Strip column names
    monthly.columns = monthly.columns.str.strip()
    gha.columns = gha.columns.str.strip()

    # Select needed columns from GHA (extend as needed)
    gha_subset = gha[
        [
            EMP_ID_COL,
            EMP_NAME_COL,
            EMP_EMAIL_COL,
            GCB_COL,
            "Company",
            "Department",
            "Job Function",
            "Legal Entity Name",
            "Employee Status"
        ]
    ].drop_duplicates()

    # Merge monthly + gha
    merged = monthly.merge(
        gha_subset,
        on=EMP_ID_COL,
        how="left",
        suffixes=("", "_GHA")
    )

    # Add Manager GCB by merging again on Manager ID
    mgr_gcb = gha_subset[[EMP_ID_COL, GCB_COL]].rename(
        columns={EMP_ID_COL: MGR_ID_COL, GCB_COL: "Direct Manager GCB"}
    )
    merged = merged.merge(mgr_gcb, on=MGR_ID_COL, how="left")

    # Capture missing GHA matches
    missing = merged[merged[GCB_COL].isna()]

    # Save Phase 1
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        merged.to_excel(writer, index=False, sheet_name="Enriched")
        if not missing.empty:
            missing.to_excel(writer, index=False, sheet_name="Missing in GHA")

    print(f"✅ Phase 1 completed → {output_file}")
    return merged


# -------------------------
# Phase 2: Build Hierarchy
# -------------------------
def build_hierarchy(enriched_df, output_file="phase2_hierarchy.xlsx"):
    """Build flattened hierarchy with MD → GCB3 → Managers → Employees."""

    df_lookup = enriched_df.set_index(EMP_ID_COL)

    final_rows = []
    exceptions = []

    def get_emp(emp_id):
        try:
            return df_lookup.loc[emp_id]
        except KeyError:
            return None

    # Find all MDs
    mds = enriched_df[enriched_df[GCB_COL] == "MD"]

    # Process each MD
    for _, md in mds.iterrows():
        md_id = md[EMP_ID_COL]

        # Find GCB3s under MD
        gcb3s = enriched_df[(enriched_df[MGR_ID_COL] == md_id) & (enriched_df[GCB_COL] == 3)]

        if gcb3s.empty:
            # Employees directly under MD
            direct_emps = enriched_df[(enriched_df[MGR_ID_COL] == md_id) & (enriched_df[GCB_COL] != 3)]
            for _, emp in direct_emps.iterrows():
                row = {
                    "MD Name": md[EMP_NAME_COL],
                    "MD ID": md[EMP_ID_COL],
                    "GCB3 Name": "",
                    "Direct Manager Name": emp[MGR_NAME_COL],
                }
                row.update(emp.to_dict())   # keep all monthly+gha cols
                final_rows.append(row)
            continue

        # Process each GCB3
        for _, gcb3 in gcb3s.iterrows():
            gcb3_id = gcb3[EMP_ID_COL]

            # Add GCB3’s own row
            row = {
                "MD Name": md[EMP_NAME_COL],
                "MD ID": md[EMP_ID_COL],
                "GCB3 Name": gcb3[EMP_NAME_COL],
                "Direct Manager Name": gcb3[MGR_NAME_COL],
            }
            row.update(gcb3.to_dict())
            final_rows.append(row)

            # Managers (GCB3/4) under this GCB3
            rm_level = enriched_df[enriched_df[MGR_ID_COL] == gcb3_id]
            for _, rm in rm_level.iterrows():
                rm_id = rm[EMP_ID_COL]

                row = {
                    "MD Name": md[EMP_NAME_COL],
                    "MD ID": md[EMP_ID_COL],
                    "GCB3 Name": gcb3[EMP_NAME_COL],
                    "Direct Manager Name": rm[MGR_NAME_COL],
                }
                row.update(rm.to_dict())
                final_rows.append(row)

                # Employees under this RM
                emps = enriched_df[enriched_df[MGR_ID_COL] == rm_id]
                for _, emp in emps.iterrows():
                    row = {
                        "MD Name": md[EMP_NAME_COL],
                        "MD ID": md[EMP_ID_COL],
                        "GCB3 Name": gcb3[EMP_NAME_COL],
                        "Direct Manager Name": rm[EMP_NAME_COL],
                    }
                    row.update(emp.to_dict())
                    final_rows.append(row)

    # Exceptions: employees with missing managers (excluding MDs)
    all_emp_ids = set(enriched_df[EMP_ID_COL])
    all_mgr_ids = set(enriched_df[MGR_ID_COL])
    missing_mgr_ids = all_mgr_ids - all_emp_ids

    for _, emp in enriched_df[enriched_df[MGR_ID_COL].isin(missing_mgr_ids)].iterrows():
        if emp[GCB_COL] != "MD":
            exceptions.append(emp.to_dict())

    # Convert to DataFrames
    final_df = pd.DataFrame(final_rows)
    exceptions_df = pd.DataFrame(exceptions)

    # Sorting
    final_df.sort_values(
        by=["MD Name", "GCB3 Name", "Direct Manager Name", EMP_NAME_COL],
        inplace=True,
        na_position="last"
    )

    # -------------------------
    # Build summary sheet
    # -------------------------
    summary_data = {
        "Metric": [
            "Total employees in enriched file",
            "Employees missing in GHA",
            "Employees with missing manager (non-MD)"
        ],
        "Count": [
            len(enriched_df),
            sum(enriched_df[GCB_COL].isna()),
            len(exceptions_df)
        ]
    }
    summary_df = pd.DataFrame(summary_data)

    # Per MD
    per_md = final_df.groupby("MD Name")[EMP_ID_COL].nunique().reset_index(name="Employees under MD")
    # Per GCB3
    per_gcb3 = final_df.groupby("GCB3 Name")[EMP_ID_COL].nunique().reset_index(name="Employees under GCB3")

    # Save
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        final_df.to_excel(writer, index=False, sheet_name="Hierarchy Report")
        if not exceptions_df.empty:
            exceptions_df.to_excel(writer, index=False, sheet_name="Missing Managers")
        summary_df.to_excel(writer, index=False, sheet_name="Summary")
        per_md.to_excel(writer, index=False, sheet_name="Summary", startrow=len(summary_df)+2)
        per_gcb3.to_excel(writer, index=False, sheet_name="Summary", startrow=len(summary_df)+len(per_md)+5)

    print(f"✅ Phase 2 completed → {output_file}")
    return final_df, exceptions_df, summary_df


In [ ]:
import pandas as pd

# -------------------------
# Constants
# -------------------------
EMP_ID_COL = "Employee ID"
EMP_NAME_COL = "Employee Name"
EMP_EMAIL_COL = "Employee Business Email Address"
MGR_ID_COL = "Entity Manager Employee ID"   # must exist in monthly.csv
MGR_NAME_COL = "Entity Manager Employee Name"
GCB_COL = "Global Career Band"

# -------------------------
# Phase 1: Enrichment
# -------------------------
def enrich_monthly_with_gha(monthly_file, gha_file, output_file="phase1_enriched.xlsx"):
    """Enrich monthly CSV with GHA details."""

    # Read monthly file (CSV may have special encoding)
    monthly = pd.read_csv(monthly_file, encoding="ISO-8859-1")
    gha = pd.read_excel(gha_file, sheet_name="Headcount - Employee Detail")

    # Strip column names
    monthly.columns = monthly.columns.str.strip()
    gha.columns = gha.columns.str.strip()

    # Select needed columns from GHA (add more here if needed)
    gha_subset = gha[
        [
            EMP_ID_COL,
            EMP_NAME_COL,
            EMP_EMAIL_COL,
            GCB_COL,
            "Company",
            "Department",
            "Job Function",
            "Legal Entity Name",
            "Employee Status"
        ]
    ].drop_duplicates()

    # Merge monthly + gha
    merged = monthly.merge(
        gha_subset,
        on=EMP_ID_COL,
        how="left",
        suffixes=("", "_GHA")
    )

    # Add Manager GCB by merging again on Manager ID
    mgr_gcb = gha_subset[[EMP_ID_COL, GCB_COL]].rename(
        columns={EMP_ID_COL: MGR_ID_COL, GCB_COL: "Manager GCB"}
    )
    merged = merged.merge(mgr_gcb, on=MGR_ID_COL, how="left")

    # Capture missing GHA matches
    missing = merged[merged[GCB_COL].isna()]

    # Save Phase 1
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        merged.to_excel(writer, index=False, sheet_name="Enriched")
        if not missing.empty:
            missing.to_excel(writer, index=False, sheet_name="Missing in GHA")

    print(f"✅ Phase 1 completed → {output_file}")
    return merged


# -------------------------
# Phase 2: Build Hierarchy
# -------------------------
def build_hierarchy(enriched_df, output_file="phase2_hierarchy.xlsx"):
    """Build flattened hierarchy with MD → GCB3 → Managers → Employees."""

    df_lookup = enriched_df.set_index(EMP_ID_COL)

    final_rows = []
    exceptions = []

    def get_emp(emp_id):
        try:
            return df_lookup.loc[emp_id]
        except KeyError:
            return None

    # Find all MDs
    mds = enriched_df[enriched_df[GCB_COL] == "MD"]

    # Process each MD
    for _, md in mds.iterrows():
        md_id = md[EMP_ID_COL]

        # Find GCB3s under MD
        gcb3s = enriched_df[(enriched_df[MGR_ID_COL] == md_id) & (enriched_df[GCB_COL] == 3)]

        if gcb3s.empty:
            # Employees directly under MD
            direct_emps = enriched_df[(enriched_df[MGR_ID_COL] == md_id) & (enriched_df[GCB_COL] != 3)]
            for _, emp in direct_emps.iterrows():
                final_rows.append({
                    "MD Name": md[EMP_NAME_COL],
                    "MD ID": md[EMP_ID_COL],
                    "GCB3 Name": "",
                    "Reporting Manager Name": emp[MGR_NAME_COL],
                    "Reporting Manager GCB": emp.get("Manager GCB", ""),
                    **emp.to_dict()
                })
            continue

        # Process each GCB3
        for _, gcb3 in gcb3s.iterrows():
            gcb3_id = gcb3[EMP_ID_COL]

            # Add GCB3’s own row
            final_rows.append({
                "MD Name": md[EMP_NAME_COL],
                "MD ID": md[EMP_ID_COL],
                "GCB3 Name": gcb3[EMP_NAME_COL],
                "Reporting Manager Name": gcb3[MGR_NAME_COL],
                "Reporting Manager GCB": gcb3.get("Manager GCB", ""),
                **gcb3.to_dict()
            })

            # Managers (GCB3/4) under this GCB3
            rm_level = enriched_df[enriched_df[MGR_ID_COL] == gcb3_id]
            for _, rm in rm_level.iterrows():
                rm_id = rm[EMP_ID_COL]

                final_rows.append({
                    "MD Name": md[EMP_NAME_COL],
                    "MD ID": md[EMP_ID_COL],
                    "GCB3 Name": gcb3[EMP_NAME_COL],
                    "Reporting Manager Name": rm[EMP_NAME_COL],
                    "Reporting Manager GCB": rm.get(GCB_COL, ""),
                    **rm.to_dict()
                })

                # Employees under this RM
                emps = enriched_df[enriched_df[MGR_ID_COL] == rm_id]
                for _, emp in emps.iterrows():
                    final_rows.append({
                        "MD Name": md[EMP_NAME_COL],
                        "MD ID": md[EMP_ID_COL],
                        "GCB3 Name": gcb3[EMP_NAME_COL],
                        "Reporting Manager Name": rm[EMP_NAME_COL],
                        "Reporting Manager GCB": rm.get(GCB_COL, ""),
                        **emp.to_dict()
                    })

    # Exceptions: employees with missing managers (excluding MDs)
    all_emp_ids = set(enriched_df[EMP_ID_COL])
    all_mgr_ids = set(enriched_df[MGR_ID_COL])
    missing_mgr_ids = all_mgr_ids - all_emp_ids

    for _, emp in enriched_df[enriched_df[MGR_ID_COL].isin(missing_mgr_ids)].iterrows():
        if emp[GCB_COL] != "MD":
            exceptions.append(emp.to_dict())

    # Convert to DataFrames
    final_df = pd.DataFrame(final_rows)
    exceptions_df = pd.DataFrame(exceptions)

    # Sorting
    final_df.sort_values(
        by=["MD Name", "GCB3 Name", "Reporting Manager Name", EMP_NAME_COL],
        inplace=True,
        na_position="last"
    )

    # Save
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        final_df.to_excel(writer, index=False, sheet_name="Hierarchy Report")
        if not exceptions_df.empty:
            exceptions_df.to_excel(writer, index=False, sheet_name="Missing Managers")

    print(f"✅ Phase 2 completed → {output_file}")
    return final_df, exceptions_df


# -------------------------
# Run pipeline
# -------------------------
if __name__ == "__main__":
    monthly_file = "monthly.csv"
    gha_file = "gha.xlsx"

    enriched = enrich_monthly_with_gha(monthly_file, gha_file)
    build_hierarchy(enriched)


In [ ]:
import pandas as pd

# -------------------------
# Phase 1: Enrichment
# -------------------------
def phase1_enrich(monthly_file, gha_file, output_file="phase1_enriched.xlsx"):
    # Read files
    monthly_df = pd.read_csv(monthly_file)
    gha_df = pd.read_excel(gha_file, sheet_name="Headcount - Employee Detail")

    # Clean column names
    monthly_df.columns = monthly_df.columns.str.strip()
    gha_df.columns = gha_df.columns.str.strip()

    # Select needed GHA columns
    gha_keep = [
        "Employee ID",
        "Employee Name",
        "Employee Business Email Address",
        "Global Career Band",
        "Legal Entity"
    ]
    gha_df = gha_df[gha_keep]

    # Merge
    merged_df = monthly_df.merge(
        gha_df,
        on="Employee ID",
        how="left",
        suffixes=("", "_GHA")
    )

    # Capture missing matches
    missing = merged_df[merged_df["Global Career Band"].isna()]

    # Save
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        merged_df.to_excel(writer, index=False, sheet_name="Enriched")
        if not missing.empty:
            missing.to_excel(writer, index=False, sheet_name="Missing in GHA")

    print(f"✅ Phase 1 done. Saved to {output_file}")
    return merged_df


# -------------------------
# Phase 2: Flattened Hierarchy
# -------------------------
def phase2_hierarchy(enriched_df, output_file="phase2_hierarchy.xlsx"):
    ID_COL = "Employee ID"
    NAME_COL = "Employee Name"
    EMAIL_COL = "Employee Business Email Address"
    MGR_ID_COL = "Manager Employee ID"
    GCB_COL = "Global Career Band"

    # Lookup for employees
    df_lookup = enriched_df.set_index(ID_COL)

    final_rows = []
    exceptions = []

    def get_emp(emp_id):
        try:
            return df_lookup.loc[emp_id]
        except KeyError:
            return None

    # Find all MDs
    mds = enriched_df[enriched_df[GCB_COL] == "MD"]

    # Process each MD
    for _, md in mds.iterrows():
        md_id = md[ID_COL]

        # Find GCB3s under MD
        gcb3s = enriched_df[(enriched_df[MGR_ID_COL] == md_id) & (enriched_df[GCB_COL] == 3)]

        if gcb3s.empty:
            # Employees directly under MD
            direct_emps = enriched_df[(enriched_df[MGR_ID_COL] == md_id) & (enriched_df[GCB_COL] != 3)]
            for _, emp in direct_emps.iterrows():
                final_rows.append({
                    "MD Name": md[NAME_COL],
                    "MD ID": md[ID_COL],
                    "MD Email": md[EMAIL_COL],
                    "GCB3 Name": "",
                    "GCB3 ID": "",
                    "GCB3 Email": "",
                    "Reporting Manager Name": "",
                    "Reporting Manager ID": "",
                    "Reporting Manager Email": "",
                    **emp.to_dict()
                })
            continue

        for _, gcb3 in gcb3s.iterrows():
            gcb3_id = gcb3[ID_COL]

            # GCB3’s own row
            final_rows.append({
                "MD Name": md[NAME_COL],
                "MD ID": md[ID_COL],
                "MD Email": md[EMAIL_COL],
                "GCB3 Name": gcb3[NAME_COL],
                "GCB3 ID": gcb3[ID_COL],
                "GCB3 Email": gcb3[EMAIL_COL],
                "Reporting Manager Name": "",
                "Reporting Manager ID": "",
                "Reporting Manager Email": "",
                **gcb3.to_dict()
            })

            # Managers (GCB3/4) under this GCB3
            rm_level = enriched_df[enriched_df[MGR_ID_COL] == gcb3_id]
            for _, rm in rm_level.iterrows():
                rm_id = rm[ID_COL]

                final_rows.append({
                    "MD Name": md[NAME_COL],
                    "MD ID": md[ID_COL],
                    "MD Email": md[EMAIL_COL],
                    "GCB3 Name": gcb3[NAME_COL],
                    "GCB3 ID": gcb3[ID_COL],
                    "GCB3 Email": gcb3[EMAIL_COL],
                    "Reporting Manager Name": rm[NAME_COL],
                    "Reporting Manager ID": rm[ID_COL],
                    "Reporting Manager Email": rm[EMAIL_COL],
                    **rm.to_dict()
                })

                # Employees under this RM
                emps = enriched_df[enriched_df[MGR_ID_COL] == rm_id]
                for _, emp in emps.iterrows():
                    final_rows.append({
                        "MD Name": md[NAME_COL],
                        "MD ID": md[ID_COL],
                        "MD Email": md[EMAIL_COL],
                        "GCB3 Name": gcb3[NAME_COL],
                        "GCB3 ID": gcb3[ID_COL],
                        "GCB3 Email": gcb3[EMAIL_COL],
                        "Reporting Manager Name": rm[NAME_COL],
                        "Reporting Manager ID": rm[ID_COL],
                        "Reporting Manager Email": rm[EMAIL_COL],
                        **emp.to_dict()
                    })

    # Exceptions: employees with missing managers (but not MDs)
    all_emp_ids = set(enriched_df[ID_COL])
    all_mgr_ids = set(enriched_df[MGR_ID_COL])
    missing_mgr_ids = all_mgr_ids - all_emp_ids

    for _, emp in enriched_df[enriched_df[MGR_ID_COL].isin(missing_mgr_ids)].iterrows():
        if emp[GCB_COL] != "MD":
            exceptions.append(emp.to_dict())

    # Convert to DataFrames
    final_df = pd.DataFrame(final_rows)
    exceptions_df = pd.DataFrame(exceptions)

    # Sorting
    final_df.sort_values(
        by=["MD Name", "GCB3 Name", "Reporting Manager Name", NAME_COL],
        inplace=True,
        na_position="last"
    )

    # Save
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        final_df.to_excel(writer, index=False, sheet_name="Hierarchy Report")
        if not exceptions_df.empty:
            exceptions_df.to_excel(writer, index=False, sheet_name="Missing Managers")

    print(f"✅ Phase 2 done. Flattened hierarchy saved to {output_file}")
    return final_df, exceptions_df


# -------------------------
# Run pipeline
# -------------------------
if __name__ == "__main__":
    monthly_file = "monthly.csv"
    gha_file = "gha.xlsx"

    enriched = phase1_enrich(monthly_file, gha_file)
    phase2_hierarchy(enriched)


In [ ]:
import pandas as pd

# -------------------------
# Phase 1: Enrichment
# -------------------------
def phase1_enrich(monthly_file, gha_file, output_file="phase1_enriched.xlsx"):
    # Read files
    monthly_df = pd.read_csv(monthly_file)
    gha_df = pd.read_excel(gha_file, sheet_name="Headcount - Employee Detail")

    # Clean column names
    monthly_df.columns = monthly_df.columns.str.strip()
    gha_df.columns = gha_df.columns.str.strip()

    # Select needed GHA columns
    gha_keep = [
        "Employee ID",
        "Employee Name",
        "Employee Business Email Address",
        "Global Career Band",
        "Legal Entity"
    ]
    gha_df = gha_df[gha_keep]

    # Merge
    merged_df = monthly_df.merge(
        gha_df,
        on="Employee ID",
        how="left",
        suffixes=("", "_GHA")
    )

    # Capture missing matches
    missing = merged_df[merged_df["Global Career Band"].isna()]

    # Save
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        merged_df.to_excel(writer, index=False, sheet_name="Enriched")
        if not missing.empty:
            missing.to_excel(writer, index=False, sheet_name="Missing in GHA")

    print(f"✅ Phase 1 done. Saved to {output_file}")
    return merged_df


# -------------------------
# Phase 2: Flattened Hierarchy
# -------------------------
def phase2_hierarchy(enriched_df, output_file="phase2_hierarchy.xlsx"):
    ID_COL = "Employee ID"
    NAME_COL = "Employee Name"
    EMAIL_COL = "Employee Business Email Address"
    MGR_ID_COL = "Manager Employee ID"
    GCB_COL = "Global Career Band"

    # Lookup for employees
    df_lookup = enriched_df.set_index(ID_COL)

    final_rows = []
    exceptions = []

    def get_emp(emp_id):
        try:
            return df_lookup.loc[emp_id]
        except KeyError:
            return None

    # Find all MDs
    mds = enriched_df[enriched_df[GCB_COL] == "MD"]

    # Process each MD
    for _, md in mds.iterrows():
        md_id = md[ID_COL]

        # Find GCB3s under MD
        gcb3s = enriched_df[(enriched_df[MGR_ID_COL] == md_id) & (enriched_df[GCB_COL] == 3)]

        if gcb3s.empty:
            # Employees directly under MD
            direct_emps = enriched_df[(enriched_df[MGR_ID_COL] == md_id) & (enriched_df[GCB_COL] != 3)]
            for _, emp in direct_emps.iterrows():
                final_rows.append({
                    "MD Name": md[NAME_COL],
                    "MD ID": md[ID_COL],
                    "MD Email": md[EMAIL_COL],
                    "GCB3 Name": "",
                    "GCB3 ID": "",
                    "GCB3 Email": "",
                    "Reporting Manager Name": "",
                    "Reporting Manager ID": "",
                    "Reporting Manager Email": "",
                    **emp.to_dict()
                })
            continue

        for _, gcb3 in gcb3s.iterrows():
            gcb3_id = gcb3[ID_COL]

            # GCB3’s own row
            final_rows.append({
                "MD Name": md[NAME_COL],
                "MD ID": md[ID_COL],
                "MD Email": md[EMAIL_COL],
                "GCB3 Name": gcb3[NAME_COL],
                "GCB3 ID": gcb3[ID_COL],
                "GCB3 Email": gcb3[EMAIL_COL],
                "Reporting Manager Name": "",
                "Reporting Manager ID": "",
                "Reporting Manager Email": "",
                **gcb3.to_dict()
            })

            # Managers (GCB3/4) under this GCB3
            rm_level = enriched_df[enriched_df[MGR_ID_COL] == gcb3_id]
            for _, rm in rm_level.iterrows():
                rm_id = rm[ID_COL]

                final_rows.append({
                    "MD Name": md[NAME_COL],
                    "MD ID": md[ID_COL],
                    "MD Email": md[EMAIL_COL],
                    "GCB3 Name": gcb3[NAME_COL],
                    "GCB3 ID": gcb3[ID_COL],
                    "GCB3 Email": gcb3[EMAIL_COL],
                    "Reporting Manager Name": rm[NAME_COL],
                    "Reporting Manager ID": rm[ID_COL],
                    "Reporting Manager Email": rm[EMAIL_COL],
                    **rm.to_dict()
                })

                # Employees under this RM
                emps = enriched_df[enriched_df[MGR_ID_COL] == rm_id]
                for _, emp in emps.iterrows():
                    final_rows.append({
                        "MD Name": md[NAME_COL],
                        "MD ID": md[ID_COL],
                        "MD Email": md[EMAIL_COL],
                        "GCB3 Name": gcb3[NAME_COL],
                        "GCB3 ID": gcb3[ID_COL],
                        "GCB3 Email": gcb3[EMAIL_COL],
                        "Reporting Manager Name": rm[NAME_COL],
                        "Reporting Manager ID": rm[ID_COL],
                        "Reporting Manager Email": rm[EMAIL_COL],
                        **emp.to_dict()
                    })

    # Exceptions: employees with missing managers (but not MDs)
    all_emp_ids = set(enriched_df[ID_COL])
    all_mgr_ids = set(enriched_df[MGR_ID_COL])
    missing_mgr_ids = all_mgr_ids - all_emp_ids

    for _, emp in enriched_df[enriched_df[MGR_ID_COL].isin(missing_mgr_ids)].iterrows():
        if emp[GCB_COL] != "MD":
            exceptions.append(emp.to_dict())

    # Convert to DataFrames
    final_df = pd.DataFrame(final_rows)
    exceptions_df = pd.DataFrame(exceptions)

    # Sorting
    final_df.sort_values(
        by=["MD Name", "GCB3 Name", "Reporting Manager Name", NAME_COL],
        inplace=True,
        na_position="last"
    )

    # Save
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        final_df.to_excel(writer, index=False, sheet_name="Hierarchy Report")
        if not exceptions_df.empty:
            exceptions_df.to_excel(writer, index=False, sheet_name="Missing Managers")

    print(f"✅ Phase 2 done. Flattened hierarchy saved to {output_file}")
    return final_df, exceptions_df


# -------------------------
# Run pipeline
# -------------------------
if __name__ == "__main__":
    monthly_file = "monthly.csv"
    gha_file = "gha.xlsx"

    enriched = phase1_enrich(monthly_file, gha_file)
    phase2_hierarchy(enriched)


In [17]:
import pandas as pd

# Sample data
data = [
    ["101", "Alice",    "5", "201", "Bob"],
    ["201", "Bob",      "4", "301", "Charlie"],
    ["301", "Charlie",  "3", "401", "Dana"],
    ["401", "Dana",     "2", None,  None],
    ["302", "Eva",      "5", "201", "Bob"],
    ["304", "Kirti",    "5", "305", "Inish"],
    ["307", "Anshul",   "4", "305", "Inish"],
    ["308", "Sowmya",   "5", "307", "Anshul"],
    ["305", "Inish",    "3", "306", "Vilma"],
    ["306", "Vilma",    "2", "504", "Abhishek"],
]

columns = [
    "Employee ID", "Employee Name", "Global Career Band",
    "Entity Manager Employee ID", "Entity Manager Employee Name"
]

df = pd.DataFrame(data, columns=columns)

# Create lookup dictionary
employee_lookup = df.set_index("Employee ID").to_dict("index")

# Add GCB 4 and GCB 3 output columns
df["Entity Manager Employee ID_GCB 4"] = ""
df["Entity Manager Employee Name_GCB 4"] = ""
df["Entity Manager Employee ID_GCB 3"] = ""
df["Entity Manager Employee Name_GCB 3"] = ""

# 🔁 Updated logic: Climb upward until GCB 4 & 3 found
def trace_managers_gcb_4_and_3(start_id):
    gcb4 = None
    gcb3 = None
    visited = []

    current_id = start_id
    while current_id and current_id in employee_lookup:
        visited.append(current_id)
        manager = employee_lookup[current_id]
        gcb = str(manager.get("Global Career Band", "")).strip()

        if not gcb4 and gcb == "4":
            gcb4 = (current_id, manager.get("Employee Name"))
        elif not gcb3 and gcb == "3":
            gcb3 = (current_id, manager.get("Employee Name"))
        
        # Stop if both found
        if gcb4 and gcb3:
            break

        current_id = manager.get("Entity Manager Employee ID")

    return gcb4, gcb3

# Apply to each row
for i in df.index:
    row = df.loc[i]
    mgr_id = row["Entity Manager Employee ID"]

    if pd.notna(mgr_id):
        gcb4, gcb3 = trace_managers_gcb_4_and_3(mgr_id)

        if gcb4:
            df.loc[i, "Entity Manager Employee ID_GCB 4"] = gcb4[0]
            df.loc[i, "Entity Manager Employee Name_GCB 4"] = gcb4[1]

        if gcb3:
            df.loc[i, "Entity Manager Employee ID_GCB 3"] = gcb3[0]
            df.loc[i, "Entity Manager Employee Name_GCB 3"] = gcb3[1]

# ✅ Final output
print(df.to_string(index=False))


Employee ID Employee Name Global Career Band Entity Manager Employee ID Entity Manager Employee Name Entity Manager Employee ID_GCB 4 Entity Manager Employee Name_GCB 4 Entity Manager Employee ID_GCB 3 Entity Manager Employee Name_GCB 3
        101         Alice                  5                        201                          Bob                              201                                Bob                              301                            Charlie
        201           Bob                  4                        301                      Charlie                                                                                                  301                            Charlie
        301       Charlie                  3                        401                         Dana                                                                                                                                        
        401          Dana                  2        

In [18]:
df.to_excel("g1.xlsx", index=False)

In [ ]:
wor